In [1]:
import os
import bisect
import sys
import gzip
import math
import random
import glob
import operator
import re
import cython
from pprint import pprint
# from gensim.models import Word2Vec
# import fuzzywuzzy
# from fuzzywuzzy import process
# import mmh3
import matplotlib.pyplot as plt
from copy import deepcopy
from collections import defaultdict, Counter, deque
import itertools
import numpy as np
import pandas as pd
import sklearn.cluster
from sklearn.cluster import AffinityPropagation
import distance
from Bio import SeqIO
from fasta_parser import parse_fasta
from system_utils import get_csvs_to_df_merge, concatenation_dfs
#from sequence_utils import get_all_possible_kmers
#from alphabet import iupac_dna

In [8]:
def minpositive(a):
    A = set(a)
    ans = 1
    while ans in A:
        ans += 1
    return ans

In [10]:
def solution(A):
    B = set(sorted(A))
    m = 1
    for x in B:
        if x == m:
            m += 1
    return m

A binary gap within a positive integer N is any maximal sequence of consecutive zeros that is surrounded by ones at both ends in the binary representation of N.

For example, number 9 has binary representation 1001 and contains a binary gap of length 2. The number 529 has binary representation 1000010001 and contains two binary gaps: one of length 4 and one of length 3. The number 20 has binary representation 10100 and contains one binary gap of length 1. The number 15 has binary representation 1111 and has no binary gaps. The number 32 has binary representation 100000 and has no binary gaps.

Write a function:

def solution(N)

that, given a positive integer N, returns the length of its longest binary gap. The function should return 0 if N doesn't contain a binary gap.

For example, given N = 1041 the function should return 5, because N has binary representation 10000010001 and so its longest binary gap is of length 5. Given N = 32 the function should return 0, because N has binary representation '100000' and thus no binary gaps.

Write an efficient algorithm for the following assumptions:

N is an integer within the range [1..2,147,483,647].

In [89]:
def binary_gap(n):
    # convert int into binary
    str_bin = format(n, 'b')
    # remove leading and trailing zeros
    # and split in ones
    # then find and return the length of the longest
    # elemnet of the list
    big_gap = len(max(str_bin.strip('0').split('1')))
    return big_gap

In [91]:
binary_gap(32)

0

In [139]:
def rotate_array(lst, k):
    # get the list as a collections deque
    d = deque(lst)
    # iterates to the k elements
    try:
        for i in range(k):
            # remove and get back the last element
            pop = d.pop()
            # append at the left side the last element
            d.appendleft(pop)
        # return the array rotate k times
        return list(d)
    except:
        return lst

In [140]:
print(rotate_array([3, 8, 9, 7, 6], 3)) # [9, 7, 6, 3, 8]
print(rotate_array([0, 0, 0], 1))
print(rotate_array([1, 2, 3, 4], 4)) # [1, 2, 3, 4]
print(rotate_array([], 1)) # [1, 2, 3, 4]

[9, 7, 6, 3, 8]
[0, 0, 0]
[1, 2, 3, 4]
[]


In [134]:
def rotate_list(lst, k):
    last = lst.pop()
    for i in range(k):
        lst.append(last)
    last = lst
    return lst

In [135]:
print(rotate_list([3, 8, 9, 7, 6], 3)) # [9, 7, 6, 3, 8]
print(rotate_list([0, 0, 0], 1))
print(rotate_list([1, 2, 3, 4], 4)) # [1, 2, 3, 4]

[3, 8, 9, 7, 6, 6, 6]
[0, 0, 0]
[1, 2, 3, 4, 4, 4, 4]


In [136]:
def rotLeft(a, k):
    return a[k: len(a)] + a[0: k]

In [137]:
print(rotLeft([3, 8, 9, 7, 6], 3)) # [9, 7, 6, 3, 8]
print(rotLeft([0, 0, 0], 1))
print(rotLeft([1, 2, 3, 4], 4)) # [1, 2, 3, 4]

[7, 6, 3, 8, 9]
[0, 0, 0]
[1, 2, 3, 4]


In [2]:
A = [9, 3, 9, 3, 9, 7, 9]

In [17]:
def check_the_odd_occurency_in_list(lst):
    from itertools import groupby
    sorted_lst = sorted(lst)
    for n, l in groupby(sorted_lst):
        if len(list(l)) % 2 == 1:
            return n

In [19]:
check_the_odd_occurency_in_list([])

In [25]:
def FrogRiverOne(X, A):
    found = set()
    for pos, i in enumerate(A, 0):
        print(i)
        if i <= X:
            found.add(i)
            if len(found) == X:
                return pos
    return -1

In [26]:
A = [1, 3, 1, 4, 2, 3, 5, 4]
FrogRiverOne(5, A)

1
3
1
4
2
3
5


6

In [ ]:
  A[0] = 1
  A[1] = 3
  A[2] = 1
  A[3] = 4
  A[4] = 2
  A[5] = 3
  A[6] = 5
  A[7] = 4

In [2]:
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 50)  

In [2]:
dir_name = 'Data/Genomes_splitted'

In [3]:
names = sorted(os.listdir(dir_name))

In [4]:
def get_all_fasta(dir_in, ssubdir_in, extension):
    """
    Function to fecth all fasta files from a directory.
    The directory has a tree like:
    root/subdir/genus_name/subsub_dir. Ex: 'Data/Genomes_splitted/Salmonella/Chromosomes'.
    
    Inputs:
        dir_in - a string representing the root directory.
        name - a string representing representing the name of a subdirectory,
               that represent a species or genus.
        ssubdir_in - a string representing a subdirectory inside the root directory.
        extension - a string representing the type of the file (compressed or not). Ex., 'gz'.
    
    Outputs:
        all_fasta -  a dictionary-like object mapping the genus name (as key) to their
                    list of full path to the fasta files.
    """
    names = sorted(os.listdir(dir_name))
    # initialize the container
    all_fasta = defaultdict(list)
    # get the full paths to the fasta files
    for name in names: 
        full_path = os.path.join(dir_in, name, ssubdir_in)
        # add the names]keys and the fullpaths to the container
        all_fasta[name] = all_fasta.get(name, []) + glob.glob(full_path + f'/*.{extension}')
    return all_fasta       

In [5]:
fasta_files = get_all_fasta('Data/Genomes_splitted', 'Chromosomes', 'gz')

In [7]:
def get_pattern_count(sequence, pattern):
    """
    Return the count the input pattern found in to a give string.
    :param sequence:
    :param pattern:
    :return:
    """
    return len(re.findall(r'(?=' + pattern + ')', sequence))    

In [8]:
filename = 'Data/Genomes_splitted/Acidovorax/Chromosomes/GCF_003030325.1_chr.fna.gz'  # amb 21987
for n, seq in parse_fasta(filename):

1748113


In [9]:
%timeit amb = get_pattern_count(seq, r'[NXYZ]')

266 ms ± 12.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
def count_umbiguous_bases(sequence):
    """
    Function to count all umbigous bases in a sequence.
    Ambigous bases are bases that are not in the sequence
    alphabet, ie. 'ACGT' for DNA sequences.

    Inputs:

        sequence - a string representing a DNA sequence.

    Outputs:

        integer - represent the number of ambigous bases found and
                  counted in a sequence.
    """
    sequence = sequence.upper()
    amb = ['N', 'R', 'Y', 'W', 'S', 'K', 'M']
    return sum({base: sequence.count(base) for base in amb}.values())

In [45]:
count_umbiguous_bases(seq)

1748113

In [11]:
%timeit ab = count_umbiguous_bases(seq)

15.4 ms ± 185 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
def cleaning_ambiguous_bases(seq):
    """
    Function to clean up all umbigous bases in a sequence.
    Ambigous bases are bases that are not in the sequence
    alphabet, ie. 'ACGT' for DNA sequences.

    Inputs:

        sequence - a string representing a DNA sequence.

    Outputs:

        integer - a new clean up string representing a DNA sequence
                  without any ambiguous characteres.
    """
    # compile the regex with all ambiguous bases
    pat = re.compile(r'[NRYWXSKM]')
    # look for the ambiguous bases and replace by
    # nothing
    new_seq = re.sub(pat,  '',    seq)
    return new_seq

In [41]:
t = 'ACCAGCATTTACXNNAYYKMTGGA'

In [75]:
%timeit cleaning_ambiguous_bases(seq)

166 ms ± 11.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [42]:
cleaner(t)

'ACCAGCATTTACATGGA'

In [74]:
def check_and_clean_sequence(sequence, alphabet):
    """
    Function to check and clean up all umbigous bases in a sequence.
    Ambigous bases are bases that are not in the sequence
    alphabet, ie. 'ACGT' for DNA sequences.

    Inputs:

        sequence - a string representing a DNA sequence.

    Outputs:

        cleaned_sequence - cleaned sequence or a string.
    """    
    if set(sequence).issubset(iupac_dna):
        return sequence
    else:
        return cleaning_ambiguous_bases(sequence)

In [77]:
len(seq)/1000

5539.776

In [70]:
%timeit check_sequence(seq, iupac_dna)

200 ms ± 436 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [71]:
%timeit count_umbiguous_bases(check_sequence(seq, iupac_dna))

210 ms ± 839 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [72]:
check_sequence(t, iupac_dna)

'ACCAGCATTTACATGGA'

In [73]:
t2 = 'ACCAGCATTTACATGGA'
check_sequence(t2, iupac_dna)

'ACCAGCATTTACATGGA'

In [64]:
if set(t).issubset(iupac_dna):
    print('clean')
else:
    print('not clean')

not clean


In [56]:
def remove_ambiguous_bases(seq):
    to_remove = dict((char, '') for char in 'NRYWXSKM')
    return ''.join([base for base in seq if base not in to_remove])

In [61]:
%timeit remove_ambiguous_bases(seq)

183 ms ± 3.21 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
def sequence_cleaner(sequence, alphabet):
    """
    Clean up a sequence from not allowed characters.
    Input:
        sequence - sequence or a string
    Output:
        sequence - cleaned sequence or a string
    """
    seq = sequence.upper()
    sequence = [base for base in seq if base in alphabet]
    return ''.join(sequence)

In [14]:
%timeit sequence_cleaner(seq, iupac_dna)

290 ms ± 8.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [80]:
def count_n_grams_fasta(fasta_dict, name, kmin, kmax):
    """
    Function to count all n-grams/k-mers (substrings of lenght n or k) in a
    big string/genome.
    
    Inputs:
        fasta_dict - a dictionary-like object that map a word/kmer to their value,
                    in this case a full path to the files to be analized.
        name - a string representing a word (key) that represent a key in a
               dictionary.
        kmin - a integer representing the lower bound of the kmer/n-gram length.
        kmax - a integer representing the maximum bound of the kmer/n-gram length.
    
    Outputs:
        final_counter - a dictionary-like mapping the kmers to their calculated count
                        in the input string, from a file.
    """
    # get the number of files in the names directory
    num_fastas = len(fasta_dict[name])
    # initialyze the counter
    counter = Counter()
    # iterates through the list of paths
    for filename in fasta_dict[name]:
        # reads the file and parse the content 
        print(f'Reading and parsing the filename {filename}')
        for name, sequence in parse_fasta(filename):
            # counting the kmers
            cnt = count_kmers_cython(sequence, kmin, kmax, counter=None)
            # add the count of the current file to the counter
            counter.update(cnt)
    # to get the mean of the kmer count for all the files
    final_counter = {k:(c//num_files) for k, c in counter.items()}
    return final_counter

In [74]:
merged = count_n_grams_fasta(fasta_files, 'Acidiphilium', 3, 6)

In [79]:
merged['TTGAC']

1485

In [62]:
counter = Counter()
num_files = len(filename_lis)

for filename in filename_lis:
    print(filename)
    for name, sequence in parse_fasta(filename):
        cnt = count_kmers_cython(sequence, 3, 6, counter=None)
        counter.update(cnt)
final = {k:c//num_files for k, c in counter.items()}

'Data/Genomes_splitted/Salmonella/Chromosomes'

['Data/Genomes_splitted/Salmonella/Chromosomes']

In [37]:
l = ['Data', 'Kmer_splitted', 'Anoxybacillus', 'Chromosomes', 'kmers', 'GCF_001634285.1_chr_10.csv.gz']
f = f'Results/Kmer_merged/{l[2]}/{l[3]}'
csv_name = f'{f}/{l[2]}_{l[4]}_merg.csv'

In [39]:
print(f)
print(csv_name)

Results/Kmer_merged/Anoxybacillus/Chromosomes
Results/Kmer_merged/Anoxybacillus/Chromosomes/Anoxybacillus_kmers_merg.csv


In [14]:
filename_lis = ['Data/Genomes_splitted/Acidiphilium/Chromosomes/GCF_000016725.1_chr.fna.gz',
                'Data/Genomes_splitted/Acidiphilium/Chromosomes/GCF_000202835.1_chr.fna.gz']

In [51]:
def get_kmer_from_csv(csvs_filename, dir_out, alphabet, kmin, kmax):
    # get a list from filename
    filename_list = csvs_filename.split('/')
    # get full path to save the csv final
    # Results]Kmer_merged/genus/Chromosomes
    full_path = os.path.join(dir_out, filename_list[2], filename_list[3])
    print(full_path)
    # get csv name
    csv_name = f'{filename_list[2]}_{filename_list[4]}_merg.csv'
    print(csv_name)
    # initialyze the counter with all possible kmer with length
    # between kmin and kmax with zero counts
    counter1 = Counter(dict([(km, 0) for km in get_all_possible_kmers(alphabet, kmin, kmax)]))
    file_num = 0
    with gzip.open(csvs_filename, 'rt') as fh:
        for line in fh:
            line = line.split(',')
            if line[0] in counter:
                counter[line[0]] += int(line[1])
        file_num += 1
    dict_mean = {k: c/file_num for k, c in counter.items()}
    df = pd.DataFrame(dict_mean.items(), columns=['kmer','count'])
    print(file_num)
    if not os.path.exists(full_path):
        os.makedirs(full_path)    
    df.to_csv(f'{full_path}/{csv_name}', index=False)

In [52]:
for filename in filename_lis:
    get_kmer_from_csv(filename, 'Results/tests', 'ACGT', 3, 10)

Results/tests/Acidiphilium/Chromosomes
Acidiphilium_GCF_000016725.1_chr.fna.gz_merg.csv
1
Results/tests/Acidiphilium/Chromosomes
Acidiphilium_GCF_000202835.1_chr.fna.gz_merg.csv
1


In [220]:
def Readtoarray():
    c = Counter()
    with open('Book1.csv','r') as file:
        reader = csv.reader(file, delimiter=',')
        next(reader, None)  # skip the headers
        for row in reader:
            c[row[0]] += 1 # or whatever the row index is for the date(s)
    return c

,kmer,count
0,AAAA,3763
1,AAAC,4568
2,AAAG,3647
3,AAAT,5682
4,AACA,6388
...,...,...
1398011,TTTTTTTTGT,0
1398012,TTTTTTTTTA,1
1398013,TTTTTTTTTC,1
1398014,TTTTTTTTTG,0


In [221]:
pd.read_csv('Data/Kmer_splitted/Acidiphilium/Chromosomes/kmers/GCF_000016725.1_chr_10.csv.gz', 
            compression='gzip')

,kmer,count
0,ATG,47595
1,TGG,50604
2,GGG,59700
3,GGT,52356
4,GTT,28385
...,...,...
888063,TATTCCCTCA,1
888064,ATTCCCTCAC,1
888065,TTCCCTCACT,1
888066,TCCCTCACTT,1


In [216]:
c = Counter(names)

In [217]:
c.items()

dict_items([('Acidiphilium', 1), ('Acidipropionibacterium', 1), ('Acidisarcina', 1), ('Acidithiobacillus', 1), ('Acidobacterium', 1), ('Acidothermus', 1), ('Acidovorax', 1), ('Acinetobacter', 1), ('Actinoalloteichus', 1), ('Actinobacillus', 1), ('Actinobaculum', 1), ('Actinomadura', 1), ('Actinomyces', 1), ('Actinoplanes', 1), ('Actinopolymorpha', 1), ('Actinopolyspora', 1), ('Actinosynnema', 1), ('Actinotalea', 1), ('Actinotignum', 1), ('Acutalibacter', 1), ('Adhaeribacter', 1), ('Adlercreutzia', 1), ('Advenella', 1), ('Aequorivita', 1), ('Aeribacillus', 1), ('Aerococcus', 1), ('Aeromicrobium', 1), ('Aeromonas', 1), ('Aerosticca', 1), ('Afipia', 1), ('Agarilytica', 1), ('Agarivorans', 1), ('Aggregatibacter', 1), ('Agrobacterium', 1), ('Agrococcus', 1), ('Agromyces', 1), ('Ahniella', 1), ('Akkermansia', 1), ('Alcaligenes', 1), ('Alcanivorax', 1), ('Algibacter', 1), ('Algicoccus', 1), ('Algoriphagus', 1), ('Aliarcobacter', 1), ('Alicycliphilus', 1), ('Alicyclobacillus', 1), ('Alienimona

In [84]:
def data_generator(fasta_path_dict):
    for name, filenames in fasta_path_dict.items():
        spc = name
        for filename in filenames:
            yield spc, glob.glob(filename+'/*.csv.gz')

In [90]:
name, pwd = list(data_generator(fasta_dict))[0]

In [58]:
def get_csvs_to_df_merge(spc_name, filenames):
    dfs = []
    for filename in filenames_dict[spc_name]:
        dfs.append(pd.read_csv(filename, dtype={'kmers': str, 'count': np.int32}))
    dfs = [df.set_index("kmers", drop=True) for df in dfs]
    merged = pd.concat(dfs,
                       axis=1,
                       keys=range(len(dfs)),
                       copy=False).sort_index().mean(axis=1).astype(int).reset_index()
    return merged.rename(columns={'index': 'kmers', 0: 'counts'}), len(dfs)  

In [95]:
get_csvs_to_df_merge(name, data_generator(fasta_dict))

TypeError: 'generator' object is not subscriptable

In [4]:
%load_ext cython

In [6]:
%%cython
from collections import defaultdict
from sequence_utils import get_all_possible_kmers
from alphabet import iupac_dna


def count_kmers_cython(str string, int kmin, int kmax, counter=None):
    """
    Count occurrence of kmers in a given string.
    """
    if counter is None:
        counter = defaultdict(int, [(km, 0) for km in get_all_possible_kmers(iupac_dna,
                                                                             kmin,
                                                                             kmax)])
    cdef int i
    cdef int j
    cdef int N = len(string)
    for k in range(kmin, kmax + 1):
        for i in range(N - k + 1):
            kmer = string[i:i+k]
            counter[kmer] = counter.get(kmer, 0) + 1
    return counter

In [19]:
counter = Counter()
num_files = len(filename_lis)

for filename in filename_lis:
    print(filename)
    for name, sequence in parse_fasta(filename):
        cnt = count_kmers_cython(sequence, 3, 6, counter=None)
        counter.update(cnt)
final = {k:c//num_files for k, c in counter.items()}

Data/Genomes_splitted/Acidiphilium/Chromosomes/GCF_000016725.1_chr.fna.gz
Data/Genomes_splitted/Acidiphilium/Chromosomes/GCF_000202835.1_chr.fna.gz


In [18]:
final

{'AAA': 19091.5,
 'AAC': 30748.5,
 'AAG': 30210.5,
 'AAT': 21558.5,
 'ACA': 23653.5,
 'ACC': 57042.0,
 'ACG': 56690.0,
 'ACT': 13922.5,
 'AGA': 29869.5,
 'AGC': 64661.0,
 'AGG': 52588.5,
 'AGT': 13577.5,
 'ATA': 14129.5,
 'ATC': 73087.0,
 'ATG': 50359.5,
 'ATT': 21637.0,
 'CAA': 28467.0,
 'CAC': 48821.5,
 'CAG': 63952.5,
 'CAT': 50390.5,
 'CCA': 54573.0,
 'CCC': 67250.5,
 'CCG': 160848.0,
 'CCT': 55050.5,
 'CGA': 115380.0,
 'CGC': 193346.0,
 'CGG': 156960.0,
 'CGT': 56931.5,
 'CTA': 10771.0,
 'CTC': 58217.5,
 'CTG': 65121.5,
 'CTT': 31702.5,
 'GAA': 48961.0,
 'GAC': 59759.5,
 'GAG': 56335.0,
 'GAT': 72943.0,
 'GCA': 72320.0,
 'GCC': 160186.0,
 'GCG': 189132.0,
 'GCT': 65727.0,
 'GGA': 52185.0,
 'GGC': 156114.0,
 'GGG': 62944.5,
 'GGT': 55289.5,
 'GTA': 11535.5,
 'GTC': 60068.5,
 'GTG': 47527.5,
 'GTT': 30313.0,
 'TAA': 5090.5,
 'TAC': 11978.5,
 'TAG': 10198.5,
 'TAT': 14320.0,
 'TCA': 41085.0,
 'TCC': 53243.5,
 'TCG': 115947.5,
 'TCT': 31112.5,
 'TGA': 40564.0,
 'TGC': 73244.0,
 'TGG':

In [8]:
for n, sequence in parse_fasta('Data/Genomes_splitted/Acidiphilium/Chromosomes/GCF_000016725.1_chr.fna.gz'):
    cnt = count_kmers_cython(sequence, 2, 10, counter=None)

In [9]:
cnt

defaultdict(int,
            {'AA': 95013,
             'AC': 142883,
             'AG': 151989,
             'AT': 150222,
             'CA': 181260,
             'CC': 322931,
             'CG': 500192,
             'CT': 156877,
             'GA': 225218,
             'GC': 466006,
             'GG': 310696,
             'GT': 141248,
             'TA': 38616,
             'TC': 229440,
             'TG': 180291,
             'TT': 96344,
             'AAA': 17660,
             'AAC': 28903,
             'AAG': 28285,
             'AAT': 20164,
             'ACA': 22260,
             'ACC': 54197,
             'ACG': 53554,
             'ACT': 12872,
             'AGA': 27902,
             'AGC': 61329,
             'AGG': 50140,
             'AGT': 12618,
             'ATA': 13221,
             'ATC': 69251,
             'ATG': 47595,
             'ATT': 20155,
             'CAA': 26465,
             'CAC': 46282,
             'CAG': 60692,
             'CAT': 47821,
             '

In [13]:
sequence.count('AACTT')

697

In [13]:
%load_ext Cython

In [190]:
%%cython
import math
def compute_stats_grouped_kmers_cython(kmer_counts_merged, kmer_expec_merged, len_genome, max_e):
    """
    This function calculates the statistics like z-scores and e-values from grouped 
    kmer/palindromes values.
    Inputs:
        kmer_counts_merged - dictionary-like object with grouped counts kmers/palindromes values
                             of grouped mers.
        kmer_expec_merged - dictionary-like object with grouped expected kmers/palindromes values
                             grouped grouped mers.
        len_genome - length of the string or genome.
        max_e - float number determinating a e-value of cut off 
    Outputs:
        results - a list of lists where results[i] is of the form 
                  [k-mer, observed count, expected count, z-score, e-value].    
    """

    results = []
    # number of tests, used to convert p-value to e-value.
    kmers = list(kmer_counts_merged.keys())
    n = len(kmers)
    for kmer in kmers:
        observed = kmer_counts_merged[kmer]
        expected = kmer_expec_merged[kmer]
        sigma = math.sqrt(expected * (1 - expected / (len_genome - len(kmer) + 1)))
        if sigma == 0.0:
            z_score = 0.0
        else:
            z_score = ((kmer_counts_merged[kmer] - kmer_expec_merged[kmer]) / sigma)
        # pvalue for all kmers/palindromes under represented
        p_value_under = (math.erfc(-z_score / math.sqrt(2)) / 2)
        # pvalue for all kmers/palindromes over represented
        p_value_over = (math.erfc(z_score / math.sqrt(2)) / 2)
        # evalue for all kmers/palindromes under represented
        e_value_under = (n * p_value_under)
        # evalue for all kmers/palindromes over represented
        e_value_over = (n * p_value_over)
        if e_value_under <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_under, e_value_under])
        elif e_value_over <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_over, e_value_over])
    return results

In [30]:
import time
time.time()

1623088563.7458389

In [ ]:
compute_stats_grouped_kmers(cv, kmer_expec_merged, len_genome, max_e)

In [95]:
counter_all = Counter()
for name, seq in fasta_dict.items():
    num_seq = 
    cnts = Counter(count_kmers_cython(seq_cleaner(seq, 'ACGT'), 6, 6))
    counter_all.update(cnts)
final = {k: cnt/sum(counter)}

Counter({'CGCCGC': 16424, 'GCGGCG': 15142, 'GCGCCG': 13666, 'GCCGCC': 13378, 'CGGCGC': 13308, 'GGCGGC': 12650, 'CCGCCG': 12360, 'GCCGGC': 12291, 'CGGCGG': 11803, 'CGCCGG': 11659, 'CCGGCG': 11542, 'GCCGCG': 10498, 'CGCGCC': 10423, 'CCGCGC': 10339, 'GGCGCG': 10191, 'GCGCGC': 10167, 'CGCGGC': 10128, 'TCGCCG': 10124, 'GCGCGG': 9837, 'CGGCGA': 9427, 'CGCGCG': 9063, 'CGCCGA': 8716, 'TCGGCG': 8514, 'CGGCCG': 7941, 'GGCGCC': 7352, 'CGCCCG': 6956, 'CCGGCC': 6943, 'GGCCGG': 6788, 'CGGGCG': 6585, 'CTCGCC': 6581, 'CTCGGC': 6358, 'GCCGAG': 6344, 'GGCCGC': 6289, 'GCCCGC': 6228, 'GCGGCC': 6141, 'CCGCCC': 6094, 'CCCGGC': 5972, 'GCGCGA': 5939, 'GGCGAG': 5899, 'CAGCGC': 5854, 'GCGGGC': 5845, 'ATCGCC': 5829, 'GCGCTG': 5783, 'GCCGGG': 5771, 'GGCGAT': 5764, 'GCCTCG': 5749, 'GGGCGG': 5695, 'TCGCGC': 5665, 'CGAGGC': 5608, 'CCCGCC': 5585, 'CGATCG': 5557, 'GATCGC': 5438, 'GCGATC': 5418, 'TGCCGC': 5263, 'TGCCGG': 5245, 'CCGGCA': 5226, 'GGCGGG': 5197, 'GCGGCA': 5165, 'TCGGCC': 5154, 'GGCCGA': 5127, 'GCCGAT': 512

In [16]:
%%cython
from collections import defaultdict
def count_kmers_cython(str string, int kmin, int kmax, counter=None):
    """
    Count occurrence of kmers in a given string.
    """
    if counter is None:
        counter = defaultdict(int)
    cdef int i
    cdef int j
    cdef int N = len(string)
    for k in range(kmin, kmax + 1):
        for i in range(N - k + 1):
            kmer = string[i:i+k]
            counter[kmer] = counter.get(kmer, 0) + 1
    return counter

In [20]:
def seq_cleaner(sequence, alphabet):
    """
    Clean up a sequence from not allowed characters.
    Input:
        sequence - sequence or a string
    Output:
        sequence - cleaned sequence or a string
    """
    seq = sequence.upper()
    sequence = [base for base in seq if base in alphabet]
    return ''.join(sequence)

In [101]:
d = defaultdict(dict)
for name, seq in fasta_dict.items():
    print(name)
    d[name] = count_kmers_cython(seq_cleaner(seq, 'ACGT'), 6, 6)

NC_009484.1
NC_015186.1


In [124]:
d['NC_009484.1']['GTTAAC'], d['NC_015186.1']['GTTAAC'], 67+74, (67+74)//2

(67, 74, 141, 70)

In [96]:
cnts['GTTAAC']

74

In [ ]:
seq_dict, num_seq = seq_to_dict(dict_paths)

In [136]:
final, total_len = final_kmer_counts(seq_dict, num_seq, 'ACGT', 6, 6)

In [140]:
total_len

7138638

In [137]:
final

{'ATGGGT': 409,
 'TGGGTT': 216,
 'GGGTTG': 727,
 'GGTTGG': 621,
 'GTTGGT': 446,
 'TTGGTC': 495,
 'TGGTCA': 958,
 'GGTCAA': 590,
 'GTCAAT': 309,
 'TCAATC': 396,
 'CAATCA': 339,
 'AATCAT': 290,
 'ATCATC': 1852,
 'TCATCT': 678,
 'CATCTG': 949,
 'ATCTGA': 390,
 'TCTGAT': 371,
 'CTGATT': 319,
 'TGATTT': 354,
 'GATTTA': 87,
 'ATTTAA': 57,
 'TTTAAT': 67,
 'TTAATG': 50,
 'TAATGG': 144,
 'AATGGG': 321,
 'ATGGGC': 1278,
 'TGGGCA': 726,
 'GGGCAA': 1013,
 'GGCAAA': 546,
 'GCAAAT': 321,
 'CAAATT': 198,
 'AAATTT': 217,
 'AATTTT': 248,
 'ATTTTT': 299,
 'TTTTTA': 98,
 'TTTTAA': 52,
 'TTTAAA': 40,
 'TTAAAG': 32,
 'TAAAGA': 84,
 'AAAGAT': 264,
 'AAGATG': 593,
 'AGATGC': 1017,
 'GATGCA': 911,
 'ATGCAC': 572,
 'TGCACA': 338,
 'GCACAT': 666,
 'CACATT': 197,
 'ACATTA': 56,
 'CATTAT': 150,
 'ATTATA': 75,
 'TTATAT': 73,
 'TATATC': 241,
 'ATATCA': 229,
 'TATCAG': 249,
 'ATCAGC': 2510,
 'TCAGCA': 1380,
 'CAGCAA': 618,
 'AGCAAA': 217,
 'GCAAAA': 357,
 'CAAAAA': 270,
 'AAAAAA': 273,
 'AAAAAT': 334,
 'AAAATC': 485

In [138]:
final['GTTAAC']

70

In [97]:
counted = Counter()
for name, seq in fasta_dict.items():
    counted.update(count_kmers_cython(seq_cleaner(seq, 'ACGT'), 6, 6))

In [98]:
counted['GTTAAC']

141

In [21]:
Hi_counts = count_kmers_cython(seq_cleaner(sequence, 'ACGT'), 1, 16)

In [16]:
f = count_mers_cython(clean_seq, 1, 6)

In [17]:
f['GTTAAC']

19

In [32]:
operator.eq(cv, f)

True

In [39]:
cc['TTATAA'], f['TTATAA'], cc['GTTAAC'], f['GTTAAC']

(1776, 888, 38, 19)

In [38]:
888*2

1776

In [18]:
def get_expected_kmer_values(mer_list, kmer_counts):
    """
    
    """
    expected = defaultdict(int)
    for kmer in mer_list:
        expected[kmer] = expected.get(kmer, 0)
        prefix, sufix, center = kmer_counts[kmer[:-1]], kmer_counts[kmer[1:]], kmer_counts[kmer[1:-1]]
        if center == 0:
            expected[kmer] = 0
        else:
            expected[kmer] += int((prefix * sufix) / center)
    return expected

In [75]:
p = list(get_palindromes('ACGT', 16, 16))

In [79]:
'AAAAAAAATTTTTTT' in p

False

In [76]:
e = get_expected_kmer_values(p, cv)

KeyError: 'AAAAAAAATTTTTTT'

In [89]:
cv['GTTAAC'], e['GTTAAC']

(19, 336)

In [37]:
def is_palindrome(string):    
    '''    '''
    k, mid = len(string), len(string) // 2
    # checking even palindromes
    if k % 2 == 0:
        return string[:mid] == get_reverse_complement(string[mid:])
    # checking odd palindromes
    else:
        return string[:mid] == get_reverse_complement(string[mid+1:])

In [38]:
is_palindrome('GTTTAAAC')

True

In [39]:
def get_all_possible_kmers(alphabet, min_k, max_k):
    """Returns a list of all possible combinations of k-mers of
    length k from a input alphabet."""
    kmers = [''.join(letters) for n in range(min_k, max_k + 1)
             for letters in itertools.product(alphabet, repeat=n)]
    return kmers

In [23]:
is_palindrome('AAATTT'), is_palindrome('ATACCG')

(True, False)

In [181]:
def insert_wild_card_char(word, num_n):
    mid = len(word)//2
    hal_n = num_n // 2
    # even palindromes and even num of wild cards
    if num_n % 2 == 0 and is_palindrome(word) and len(word) % 2 == 0 and num_n < len(word):
        return word[:mid-hal_n] + ('N' * num_n) + word[mid+hal_n:], word
    # odd words and odd wild cards 
    if num_n % 2 != 0 and is_palindrome(word) and len(word) % 2 != 0 and num_n < len(word):
        return  word[:mid-hal_n] + ('N' * num_n) + word[mid+hal_n+1:], word
    else:
        return word

In [182]:
insert_wild_card_char('AACGATCGTT', 10)

'AACGATCGTT'

In [111]:
10-4, 6//2

(6, 3)

In [148]:
s = 'AACCGTT'
l = len('AACCGTT')
m = l//2
n = 4
l_n = l-n
n_2 = n // 2

In [138]:
l, n, l_n, n_2

(7, 3, 4, 1)

In [149]:
s[:m-n_2] + 'n'*n + s[m+n_2+1:]  #+'n'*n+s[m+l_n:]

'AnnnnT'

In [90]:
def wild_card_insert(word, num_n):
    mid = len(word)//2
    if num_n == 1 and is_palindrome(word) and len(word) % 2 != 0:
        return word[:mid] + 'N' + word[mid+1:], word
    # the even words can receive two wild card chars
    if num_n == 2 and is_palindrome(word) and len(word) % 2 == 0:
            return  word[:mid-1] + 'NN' + word[mid+1:], word
            # only odd words can return a word with 3 chars wild cards
    if num_n == 3 and word[:mid-1] == get_reverse_complement( word[mid+2:]) and len(word) % 2 != 0 and\
    len(word) >= 5:
            return  word[:mid-1] + 'NNN' + word[mid+2:], word
    else:
        return word

In [91]:
wild_card_insert('AACGATCGTT', num_n=1), wild_card_insert('AACGATCGTT', num_n=2), wild_card_insert('AACGATCGTT', num_n=3)

('AACGATCGTT', ('AACGNNCGTT', 'AACGATCGTT'), 'AACGATCGTT')

In [126]:
len('AACCGTT'), is_palindrome('AACCGTT')

(7, True)

In [48]:
wild_cards = [wild_card_insert(word, 2) for word in p]

In [50]:
def groupby_palindromes(wild_cards_palindrome_list):
    """
    Returns a dictionary like object where keys are palindromes
    with wild cards characters and a list of grouped
    palindromes.
    Inputs:
        palindrome_list - a list of all possible palindromes of
                          k lengths.
    Outputs:
        groups - a dictionary like object with waild cards palindromes
                 as keys and all palindromes that can match that wild
                 cards palindromes.
    Ex:
            {'AANNTT': ['AAATTT', 'AACGTT', 'AAGCTT', 'AATATT'],
             'ACNNGT': ['ACATGT', 'ACCGGT', 'ACGCGT', 'ACTAGT'],
             'AGNNCT': ['AGATCT', 'AGCGCT', 'AGGCCT', 'AGTACT'],
             'ATNNAT': ['ATATAT', 'ATCGAT', 'ATGCAT', 'ATTAAT'],
             'CANNTG': ['CAATTG', 'CACGTG', 'CAGCTG', 'CATATG'],
             'CCNNGG': ['CCATGG', 'CCCGGG', 'CCGCGG', 'CCTAGG'],
             'CGNNCG': ['CGATCG', 'CGCGCG', 'CGGCCG', 'CGTACG'],
             'CTNNAG': ['CTATAG', 'CTCGAG', 'CTGCAG', 'CTTAAG'],
             'GANNTC': ['GAATTC', 'GACGTC', 'GAGCTC', 'GATATC'],
             'GCNNGC': ['GCATGC', 'GCCGGC', 'GCGCGC', 'GCTAGC'],
             'GGNNCC': ['GGATCC', 'GGCGCC', 'GGGCCC', 'GGTACC'],
             'GTNNAC': ['GTATAC', 'GTCGAC', 'GTGCAC', 'GTTAAC'],
             'TANNTA': ['TAATTA', 'TACGTA', 'TAGCTA', 'TATATA'],
             'TCNNGA': ['TCATGA', 'TCCGGA', 'TCGCGA', 'TCTAGA'],
             'TGNNCA': ['TGATCA', 'TGCGCA', 'TGGCCA', 'TGTACA'],
             'TTNNAA': ['TTATAA', 'TTCGAA', 'TTGCAA', 'TTTAAA']}
    """
    groups = defaultdict(list)
    for key, group in itertools.groupby(wild_cards_palindrome_list, key=lambda x: x[0]):
        g_key = ''.join(key)
        groups[g_key] = groups.get(g_key, []) + [x[1] for x in group]
    return groups

In [52]:
gpal = groupby_palindromes(wild_cards)

In [62]:
gpal

defaultdict(list,
            {'AANNTT': ['AAATTT', 'AACGTT', 'AAGCTT', 'AATATT'],
             'ACNNGT': ['ACATGT', 'ACCGGT', 'ACGCGT', 'ACTAGT'],
             'AGNNCT': ['AGATCT', 'AGCGCT', 'AGGCCT', 'AGTACT'],
             'ATNNAT': ['ATATAT', 'ATCGAT', 'ATGCAT', 'ATTAAT'],
             'CANNTG': ['CAATTG', 'CACGTG', 'CAGCTG', 'CATATG'],
             'CCNNGG': ['CCATGG', 'CCCGGG', 'CCGCGG', 'CCTAGG'],
             'CGNNCG': ['CGATCG', 'CGCGCG', 'CGGCCG', 'CGTACG'],
             'CTNNAG': ['CTATAG', 'CTCGAG', 'CTGCAG', 'CTTAAG'],
             'GANNTC': ['GAATTC', 'GACGTC', 'GAGCTC', 'GATATC'],
             'GCNNGC': ['GCATGC', 'GCCGGC', 'GCGCGC', 'GCTAGC'],
             'GGNNCC': ['GGATCC', 'GGCGCC', 'GGGCCC', 'GGTACC'],
             'GTNNAC': ['GTATAC', 'GTCGAC', 'GTGCAC', 'GTTAAC'],
             'TANNTA': ['TAATTA', 'TACGTA', 'TAGCTA', 'TATATA'],
             'TCNNGA': ['TCATGA', 'TCCGGA', 'TCGCGA', 'TCTAGA'],
             'TGNNCA': ['TGATCA', 'TGCGCA', 'TGGCCA', 'TGTACA'],
       

In [79]:
# def insert_n(word, num_n=1):
#     mid = len(word)//2
#     if num_n == 1 and word[:mid] == get_reverse_complement(word[mid+1:]) and len(word) % 2 != 0:
#         return word[:mid] + 'N' + word[mid+1:], word
#     # the even words can receive two wild card chars
#     if num_n == 2 and word[:mid-1] == get_reverse_complement(word[mid+1:]) and len(word) % 2 == 0:
#             return  word[:mid-1] + 'NN' + word[mid+1:], word
#             # only odd words can return a word with 3 chars wild cards
#     if num_n == 3 and word[:mid-1] == get_reverse_complement( word[mid+2:]) and len(word) % 2 != 0 and\
#     len(word) >= 5:
#             return  word[:mid-1] + 'NNN' + word[mid+2:], word
#     else:
#         return word

In [40]:
def groupby_palindromes(wild_cards_palindrome_list):
    """
    Returns a dictionary like object where keys are palindromes
    with wild cards characters and a list of grouped
    palindromes.
    Inputs:
        palindrome_list - a list of all possible palindromes of
                          k lengths.
    Outputs:
        groups - a dictionary like object with waild cards palindromes
                 as keys and all palindromes that can match that wild
                 cards palindromes.
    Ex:
            {'AANNTT': ['AAATTT', 'AACGTT', 'AAGCTT', 'AATATT'],
             'ACNNGT': ['ACATGT', 'ACCGGT', 'ACGCGT', 'ACTAGT'],
             'AGNNCT': ['AGATCT', 'AGCGCT', 'AGGCCT', 'AGTACT'],
             'ATNNAT': ['ATATAT', 'ATCGAT', 'ATGCAT', 'ATTAAT'],
             'CANNTG': ['CAATTG', 'CACGTG', 'CAGCTG', 'CATATG'],
             'CCNNGG': ['CCATGG', 'CCCGGG', 'CCGCGG', 'CCTAGG'],
             'CGNNCG': ['CGATCG', 'CGCGCG', 'CGGCCG', 'CGTACG'],
             'CTNNAG': ['CTATAG', 'CTCGAG', 'CTGCAG', 'CTTAAG'],
             'GANNTC': ['GAATTC', 'GACGTC', 'GAGCTC', 'GATATC'],
             'GCNNGC': ['GCATGC', 'GCCGGC', 'GCGCGC', 'GCTAGC'],
             'GGNNCC': ['GGATCC', 'GGCGCC', 'GGGCCC', 'GGTACC'],
             'GTNNAC': ['GTATAC', 'GTCGAC', 'GTGCAC', 'GTTAAC'],
             'TANNTA': ['TAATTA', 'TACGTA', 'TAGCTA', 'TATATA'],
             'TCNNGA': ['TCATGA', 'TCCGGA', 'TCGCGA', 'TCTAGA'],
             'TGNNCA': ['TGATCA', 'TGCGCA', 'TGGCCA', 'TGTACA'],
             'TTNNAA': ['TTATAA', 'TTCGAA', 'TTGCAA', 'TTTAAA']}
    """
    groups = defaultdict(list)
    for key, group in itertools.groupby(wild_cards_palindrome_list, key=lambda x: x[0]):
        g_key = ''.join(key)
        groups[g_key] = groups.get(g_key, []) + [x[1] for x in group]
    return groups

In [54]:
def palindrome_group_counts(grouped_kmer_dict, kmer_counts):
    """
    Returns the merged counts of the grouped kmers/palindromes.
    Inputs:
        grouped_kmer_dict - dictionary-like object with grouped palindromes.
        kmer_counts - a dictionary-like object with kmer/palidromes observad/counted in a string
                      or genome.
    Outputs:
        counts_grouped - a dictionary-like object with kmer/palidromes merged
        count values.         
    """
    counts_grouped = defaultdict(int)
    for k, val in grouped_kmer_dict.items():
        counts_grouped[k] = counts_grouped.get(k, 0)
        for pal in val:
            counts_grouped[k] += kmer_counts[pal]
    return counts_grouped

In [65]:
cs = palindrome_group_counts(gpal, cv)

In [66]:
ce = palindrome_group_counts(gpal, e)

In [67]:
ce

defaultdict(int,
            {'AANNTT': 7363,
             'ACNNGT': 531,
             'AGNNCT': 698,
             'ATNNAT': 3553,
             'CANNTG': 1986,
             'CCNNGG': 298,
             'CGNNCG': 588,
             'CTNNAG': 993,
             'GANNTC': 1310,
             'GCNNGC': 427,
             'GGNNCC': 527,
             'GTNNAC': 923,
             'TANNTA': 2388,
             'TCNNGA': 657,
             'TGNNCA': 1567,
             'TTNNAA': 5270})

In [191]:
def compute_stats_grouped_kmers(kmer_counts_merged, kmer_expec_merged, len_genome, max_e):
    """
    This function calculates the statistics like z-scores and e-values from grouped 
    kmer/palindromes values.
    Inputs:
        kmer_counts_merged - dictionary-like object with grouped counts kmers/palindromes values
                             of grouped mers.
        kmer_expec_merged - dictionary-like object with grouped expected kmers/palindromes values
                             grouped grouped mers.
        len_genome - length of the string or genome.
        max_e - float number determinating a e-value of cut off 
    Outputs:
        results - a list of lists where results[i] is of the form 
                  [k-mer, observed count, expected count, z-score, e-value].    
    """

    results = []
    # number of tests, used to convert p-value to e-value.
    kmers = list(kmer_counts_merged.keys())
    n = len(kmers)
    for kmer in kmers:
        observed = kmer_counts_merged[kmer]
        expected = kmer_expec_merged[kmer]
        sigma = math.sqrt(expected * (1 - expected / (len_genome - len(kmer) + 1)))
        if sigma == 0.0:
            z_score = 0.0
        else:
            z_score = ((kmer_counts_merged[kmer] - kmer_expec_merged[kmer]) / sigma)
        # pvalue for all kmers/palindromes under represented
        p_value_under = (math.erfc(-z_score / math.sqrt(2)) / 2)
        # pvalue for all kmers/palindromes over represented
        p_value_over = (math.erfc(z_score / math.sqrt(2)) / 2)
        # evalue for all kmers/palindromes under represented
        e_value_under = (n * p_value_under)
        # evalue for all kmers/palindromes over represented
        e_value_over = (n * p_value_over)
        if e_value_under <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_under, e_value_under])
        elif e_value_over <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_over, e_value_over])
    return results

In [192]:
%timeit compute_stats_grouped_kmers_cython(cs, ce, seq_len, 0.01)

8.49 µs ± 454 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [193]:
%timeit md = compute_stats_grouped_kmers(cs, ce, seq_len, 0.01)

12.1 µs ± 40.9 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [53]:
def merged_stats(data, grouped_pals):
    """It returns a Data Frame like object with the final data of the script.
    Inputs:
         data - a list of lists where results[i] is of the form 
                [k-mer, observed count, expected count, z-score, e-value].
        grouped_pals - dictionary-like object with grouped kmers/palindromes.
    Outputs:
        df_final - Data Frame like object with the final data.
    """
    df1 = pd.DataFrame(grouped_pals.items(), columns=['kmer', 'groups'])
    df = pd.DataFrame(data, columns=['kmer',
                                     'count',
                                     'expected',
                                     'z_score',
                                     'p_value',
                                     'e_value']).sort_values('z_score').reset_index(drop=True)
    df_final = pd.merge(df, df1)
    df_final['group_len'] = df_final['groups'].str.len()
    return df_final

In [71]:
merged_stats(md, gpal)

,kmer,count,expected,z_score,p_value,e_value,groups,group_len
0,GTNNAC,221,923,-23.112440,1.735712e-118,2.777139e-117,"[GTATAC, GTCGAC, GTGCAC, GTTAAC]",4
1,GANNTC,577,1310,-20.259271,1.471436e-91,2.354297e-90,"[GAATTC, GACGTC, GAGCTC, GATATC]",4
2,AANNTT,6145,7363,-14.223133,3.291895e-46,5.267033e-45,"[AAATTT, AACGTT, AAGCTT, AATATT]",4
3,GGNNCC,220,527,-13.375056,4.229504e-41,6.767206e-40,"[GGATCC, GGCGCC, GGGCCC, GGTACC]",4
4,CANNTG,1394,1986,-13.291312,1.299896e-40,2.079834e-39,"[CAATTG, CACGTG, CAGCTG, CATATG]",4
5,TANNTA,1818,2388,-11.671891,8.871754e-32,1.419481e-30,"[TAATTA, TACGTA, TAGCTA, TATATA]",4
6,TTNNAA,4607,5270,-9.146068,2.952184e-20,4.723494e-19,"[TTATAA, TTCGAA, TTGCAA, TTTAAA]",4
7,ATNNAT,3019,3553,-8.967381,1.518239e-19,2.429183e-18,"[ATATAT, ATCGAT, ATGCAT, ATTAAT]",4
8,TGNNCA,1230,1567,-8.516897,8.194238e-18,1.311078e-16,"[TGATCA, TGCGCA, TGGCCA, TGTACA]",4
9,CGNNCG,412,588,-7.259284,1.945724e-13,3.113159e-12,"[CGATCG, CGCGCG, CGGCCG, CGTACG]",4


In [327]:
merged_stats(data_grouped, group_palindromes)

,kmer,count,expected,z_score,p_value,e_value,groups,group_len
0,CCNNGG,298,243,3.528486,2.089721e-04,3.343554e-03,"[CCATGG, CCCGGG, CCGCGG, CCTAGG]",4
1,GCNNGC,427,347,4.295032,8.733399e-06,1.397344e-04,"[GCATGC, GCCGGC, GCGCGC, GCTAGC]",4
2,AGNNCT,698,573,5.222768,8.813429e-08,1.410149e-06,"[AGATCT, AGCGCT, AGGCCT, AGTACT]",4
3,CTNNAG,993,775,7.832450,2.392273e-15,3.827636e-14,"[CTATAG, CTCGAG, CTGCAG, CTTAAG]",4
4,CGNNCG,588,412,8.671874,2.125333e-18,3.400533e-17,"[CGATCG, CGCGCG, CGGCCG, CGTACG]",4
5,TGNNCA,1567,1230,9.612212,3.550349e-22,5.680558e-21,"[TGATCA, TGCGCA, TGGCCA, TGTACA]",4
6,ATNNAT,3553,3019,9.726760,1.159302e-22,1.854883e-21,"[ATATAT, ATCGAT, ATGCAT, ATTAAT]",4
7,TTNNAA,5270,4607,9.780290,6.840892e-23,1.094543e-21,"[TTATAA, TTCGAA, TTGCAA, TTTAAA]",4
8,TANNTA,2388,1818,13.374998,4.232790e-41,6.772464e-40,"[TAATTA, TACGTA, TAGCTA, TATATA]",4
9,AANNTT,7363,6145,15.563837,6.408912e-55,1.025426e-53,"[AAATTT, AACGTT, AAGCTT, AATATT]",4


In [328]:
[insert_wild_card(word, 2) for word in p_list]

[('AANNTT', 'AAATTT'),
 ('AANNTT', 'AACGTT'),
 ('AANNTT', 'AAGCTT'),
 ('AANNTT', 'AATATT'),
 ('ACNNGT', 'ACATGT'),
 ('ACNNGT', 'ACCGGT'),
 ('ACNNGT', 'ACGCGT'),
 ('ACNNGT', 'ACTAGT'),
 ('AGNNCT', 'AGATCT'),
 ('AGNNCT', 'AGCGCT'),
 ('AGNNCT', 'AGGCCT'),
 ('AGNNCT', 'AGTACT'),
 ('ATNNAT', 'ATATAT'),
 ('ATNNAT', 'ATCGAT'),
 ('ATNNAT', 'ATGCAT'),
 ('ATNNAT', 'ATTAAT'),
 ('CANNTG', 'CAATTG'),
 ('CANNTG', 'CACGTG'),
 ('CANNTG', 'CAGCTG'),
 ('CANNTG', 'CATATG'),
 ('CCNNGG', 'CCATGG'),
 ('CCNNGG', 'CCCGGG'),
 ('CCNNGG', 'CCGCGG'),
 ('CCNNGG', 'CCTAGG'),
 ('CGNNCG', 'CGATCG'),
 ('CGNNCG', 'CGCGCG'),
 ('CGNNCG', 'CGGCCG'),
 ('CGNNCG', 'CGTACG'),
 ('CTNNAG', 'CTATAG'),
 ('CTNNAG', 'CTCGAG'),
 ('CTNNAG', 'CTGCAG'),
 ('CTNNAG', 'CTTAAG'),
 ('GANNTC', 'GAATTC'),
 ('GANNTC', 'GACGTC'),
 ('GANNTC', 'GAGCTC'),
 ('GANNTC', 'GATATC'),
 ('GCNNGC', 'GCATGC'),
 ('GCNNGC', 'GCCGGC'),
 ('GCNNGC', 'GCGCGC'),
 ('GCNNGC', 'GCTAGC'),
 ('GGNNCC', 'GGATCC'),
 ('GGNNCC', 'GGCGCC'),
 ('GGNNCC', 'GGGCCC'),
 ('GGNNCC',

In [333]:
def groupby_palindromes(pal_list, num_chars):
    """ 
    Returns a dictionary like object where keys are palindromes
    with wild cards characters and a list of grouped 
    palindromes.
    Inputs:
        palindrome_list - a list of all possible palindromes of 
                          k lengths.
    Outputs:
        groups - a dictionary like object with waild cards palindromes
                 as keys and all palindromes that can match that wild
                 cards palindromes.   
    Ex:
            {'AANNTT': ['AAATTT', 'AACGTT', 'AAGCTT', 'AATATT'],
             'ACNNGT': ['ACATGT', 'ACCGGT', 'ACGCGT', 'ACTAGT'],
             'AGNNCT': ['AGATCT', 'AGCGCT', 'AGGCCT', 'AGTACT'],
             'ATNNAT': ['ATATAT', 'ATCGAT', 'ATGCAT', 'ATTAAT'],
             'CANNTG': ['CAATTG', 'CACGTG', 'CAGCTG', 'CATATG'],
             'CCNNGG': ['CCATGG', 'CCCGGG', 'CCGCGG', 'CCTAGG'],
             'CGNNCG': ['CGATCG', 'CGCGCG', 'CGGCCG', 'CGTACG'],
             'CTNNAG': ['CTATAG', 'CTCGAG', 'CTGCAG', 'CTTAAG'],
             'GANNTC': ['GAATTC', 'GACGTC', 'GAGCTC', 'GATATC'],
             'GCNNGC': ['GCATGC', 'GCCGGC', 'GCGCGC', 'GCTAGC'],
             'GGNNCC': ['GGATCC', 'GGCGCC', 'GGGCCC', 'GGTACC'],
             'GTNNAC': ['GTATAC', 'GTCGAC', 'GTGCAC', 'GTTAAC'],
             'TANNTA': ['TAATTA', 'TACGTA', 'TAGCTA', 'TATATA'],
             'TCNNGA': ['TCATGA', 'TCCGGA', 'TCGCGA', 'TCTAGA'],
             'TGNNCA': ['TGATCA', 'TGCGCA', 'TGGCCA', 'TGTACA'],
             'TTNNAA': ['TTATAA', 'TTCGAA', 'TTGCAA', 'TTTAAA']}
    """
    wc = [insert_wild_card(word, num_chars) for word in p_list]
    groups = defaultdict(list)
    for key, group in itertools.groupby(wc, key=lambda x: x[0]):
        g_key = ''.join(key)
        groups[g_key] = groups.get(g_key, []) + [x[1] for x in group]
    return groups

In [334]:
groupby_palindromes(p_list, 2)

defaultdict(list,
            {'AANNTT': ['AAATTT', 'AACGTT', 'AAGCTT', 'AATATT'],
             'ACNNGT': ['ACATGT', 'ACCGGT', 'ACGCGT', 'ACTAGT'],
             'AGNNCT': ['AGATCT', 'AGCGCT', 'AGGCCT', 'AGTACT'],
             'ATNNAT': ['ATATAT', 'ATCGAT', 'ATGCAT', 'ATTAAT'],
             'CANNTG': ['CAATTG', 'CACGTG', 'CAGCTG', 'CATATG'],
             'CCNNGG': ['CCATGG', 'CCCGGG', 'CCGCGG', 'CCTAGG'],
             'CGNNCG': ['CGATCG', 'CGCGCG', 'CGGCCG', 'CGTACG'],
             'CTNNAG': ['CTATAG', 'CTCGAG', 'CTGCAG', 'CTTAAG'],
             'GANNTC': ['GAATTC', 'GACGTC', 'GAGCTC', 'GATATC'],
             'GCNNGC': ['GCATGC', 'GCCGGC', 'GCGCGC', 'GCTAGC'],
             'GGNNCC': ['GGATCC', 'GGCGCC', 'GGGCCC', 'GGTACC'],
             'GTNNAC': ['GTATAC', 'GTCGAC', 'GTGCAC', 'GTTAAC'],
             'TANNTA': ['TAATTA', 'TACGTA', 'TAGCTA', 'TATATA'],
             'TCNNGA': ['TCATGA', 'TCCGGA', 'TCGCGA', 'TCTAGA'],
             'TGNNCA': ['TGATCA', 'TGCGCA', 'TGGCCA', 'TGTACA'],
       

In [168]:
 grouped_pal = groupby_palindromes(cd)

In [110]:
# pre, suf = ''.join(list(itertools.islice(word, m))), ''.join(list(itertools.islice(word, m+1, len(word))))

In [180]:
def palindrome_group_counts(grouped_kmer_dict, kmer_counts):
    """
    
    """
    counts_grouped = defaultdict(int)
    for k, val in grouped_pal.items():
        counts_grouped[k] = counts_grouped.get(k, 0)
        for p in val:
            counts_grouped[k] += kmer_counts[p]
    return counts_grouped

In [181]:
palindrome_group_counts(grouped_pal, cv)

defaultdict(int,
            {'AANNTT': 6145,
             'ACNNGT': 476,
             'AGNNCT': 573,
             'ATNNAT': 3019,
             'CANNTG': 1394,
             'CCNNGG': 243,
             'CGNNCG': 412,
             'CTNNAG': 775,
             'GANNTC': 577,
             'GCNNGC': 347,
             'GGNNCC': 220,
             'GTNNAC': 221,
             'TANNTA': 1818,
             'TCNNGA': 662,
             'TGNNCA': 1230,
             'TTNNAA': 4607})

In [174]:
grouped_pal # GTTAAC

defaultdict(list,
            {'AANNTT': ['AAATTT', 'AACGTT', 'AAGCTT', 'AATATT'],
             'ACNNGT': ['ACATGT', 'ACCGGT', 'ACGCGT', 'ACTAGT'],
             'AGNNCT': ['AGATCT', 'AGCGCT', 'AGGCCT', 'AGTACT'],
             'ATNNAT': ['ATATAT', 'ATCGAT', 'ATGCAT', 'ATTAAT'],
             'CANNTG': ['CAATTG', 'CACGTG', 'CAGCTG', 'CATATG'],
             'CCNNGG': ['CCATGG', 'CCCGGG', 'CCGCGG', 'CCTAGG'],
             'CGNNCG': ['CGATCG', 'CGCGCG', 'CGGCCG', 'CGTACG'],
             'CTNNAG': ['CTATAG', 'CTCGAG', 'CTGCAG', 'CTTAAG'],
             'GANNTC': ['GAATTC', 'GACGTC', 'GAGCTC', 'GATATC'],
             'GCNNGC': ['GCATGC', 'GCCGGC', 'GCGCGC', 'GCTAGC'],
             'GGNNCC': ['GGATCC', 'GGCGCC', 'GGGCCC', 'GGTACC'],
             'GTNNAC': ['GTATAC', 'GTCGAC', 'GTGCAC', 'GTTAAC'],
             'TANNTA': ['TAATTA', 'TACGTA', 'TAGCTA', 'TATATA'],
             'TCNNGA': ['TCATGA', 'TCCGGA', 'TCGCGA', 'TCTAGA'],
             'TGNNCA': ['TGATCA', 'TGCGCA', 'TGGCCA', 'TGTACA'],
       

In [176]:
cv['AAATTT'], cv['AACGTT'], cv['AAGCTT'], cv['AATATT'], sum([2904, 847, 359, 2035])

(2904, 847, 359, 2035, 6145)

In [182]:
count_groups = palindrome_group_counts(grouped_pal, cv)

In [187]:
expec_groups = palindrome_group_counts(grouped_pal,e)

In [192]:
e['AAATTT'], e['AACGTT'], e['AAGCTT'], e['AATATT'], sum([3380, 923, 651, 2409])

(3380, 923, 651, 2409, 7363)

In [189]:
expec_groups

defaultdict(int,
            {'AANNTT': 7363,
             'ACNNGT': 531,
             'AGNNCT': 698,
             'ATNNAT': 3553,
             'CANNTG': 1986,
             'CCNNGG': 298,
             'CGNNCG': 588,
             'CTNNAG': 993,
             'GANNTC': 1310,
             'GCNNGC': 427,
             'GGNNCC': 527,
             'GTNNAC': 923,
             'TANNTA': 2388,
             'TCNNGA': 657,
             'TGNNCA': 1567,
             'TTNNAA': 5270})

In [91]:
def get_grouped_palindromes_hamming(pal_list, num_char=None):
    grouped_pals = defaultdict(list)
    words = np.asarray(pal_list)
    similarity = -1 * np.array([[distance.hamming(w1,w2) for 
                                     w1 in words] for w2 in words])
    
    affprop = AffinityPropagation(affinity="precomputed", 
                                  damping=0.5, 
                                  random_state=None)
    affprop.fit(similarity)
    for cluster_id in np.unique(affprop.labels_):
        exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
        cluster = np.unique(words[np.nonzero(affprop.labels_ == cluster_id)])
        if num_char != None and len(exemplar) % 2 != 0 and len(exemplar) > 3:
            grouped_pals[insert_wild_card(exemplar, num_char)[0]] = list(cluster)
        elif num_char != None and len(exemplar) % 2 == 0 and len(exemplar) > 2:
            grouped_pals[insert_wild_card(exemplar, num_char)[0]] = list(cluster)
        else:
            grouped_pals[exemplar] = list(cluster)
    return grouped_pals

In [104]:
word = 'TCGATTAATAATCGA'
m = len('TCGATTAATAATCGA')//2

In [108]:
''.join(list(itertools.islice(word, m))), ''.join(list(itertools.islice(word, m+1, len(word))))

('TCGATTA', 'TAATCGA')

In [96]:
g_h = get_grouped_palindromes_hamming(p_list, 2)

In [77]:
def get_grouped_palindromes_levenshtein(pal_list, num_char=None):
    grouped_pals = defaultdict(list)
    words = np.asarray(pal_list)
    similarity = -1 * np.array([[distance.levenshtein(w1,w2) for 
                                     w1 in words] for w2 in words])
    
    affprop = AffinityPropagation(affinity="precomputed", 
                                  damping=0.5, 
                                  random_state=None)
    affprop.fit(similarity)
    for cluster_id in np.unique(affprop.labels_):
        exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
        cluster = np.unique(words[np.nonzero(affprop.labels_ == cluster_id)])
        if num_char != None and len(exemplar) % 2 != 0 and len(exemplar) > 3:
            grouped_pals[insert_wild_card(exemplar, num_char)[0]] = list(cluster)
        elif num_char != None and len(exemplar) % 2 == 0 and len(exemplar) > 2:
            grouped_pals[insert_wild_card(exemplar, num_char)[0]] = list(cluster)
        else:
            grouped_pals[exemplar] = list(cluster)
    return grouped_pals

In [203]:
g_l = get_grouped_palindromes_levenshtein(p_list, 2)

In [198]:
operator.eq(g_l, g_h)

False

In [76]:
def merge_counts(grouped_pal_dict, kmer_counts):
    merged = defaultdict(int)
    # iterates through keys and values in the dict of grouped palindromes
    for g, pals in grouped_pal_dict.items():
        # iterates through the values lists (grouped kmers)
        for pal in pals:
            # sum up all the grouped palindromes counts
            merged[g] += kmer_counts[pal]
    return merged

In [112]:
cm = merge_counts(g, c)

In [113]:
em = merge_counts(g, e)

In [80]:
def stats_kmer(kmer_list, kmer_counts, len_genome, max_e, num_n, all_kmers=True):
    """
    This function compute_stats computes the e-values for the supplied data.
    Inputs:
        kmer_list - list of kmers (for which stats will be produced)
        kmer_counts - dictionary-like object with k-mers as keys (min_k - 2 <= k <= max_k,
                      where min_k and max_k are the bounds on the k-mer lengths in 'kmer_list')
        len_genome - length of the sequence(s) read to produce 'counts'
        max_e - the upper bound on e-values reported
        num_n - number of wild card characters to insert in the odd palindromes to allow to merge
                the counts.
        all_kmers - boolean value to assure the stats for all kmers and not only palindromes

    Outputs:
        list - list of lists ('results'), ie., [k-mer, observed count, expected count, z-score, e-value]
        grouped - a dictionary like object to check if the merged results are corrects. Ex.,
                  {'AANTT': ['AAATT', 'AACTT', 'AAGTT', 'AATTT'],
                  'ACNGT': ['ACAGT', 'ACCGT', 'ACGGT', 'ACTGT'],
                  'AGNCT': ['AGACT', 'AGCCT', 'AGGCT', 'AGTCT'],
                  'ATNAT': ['ATAAT', 'ATCAT', 'ATGAT', 'ATTAT'],...}
    """
    results = []
    #number of tests, used to convert p-value to e-value.
    n = len(kmer_list)
    for kmer in kmer_list:
        k = len(kmer)
        if k % 2 != 0 and not all_kmers:
            expec_dict = get_expected_kmer_values(kmer_list, kmer_counts)
            print(f'The length of expected values for odd palindromes is {len(expec_dict)}')
            grouped_mers = get_grouped_palindromes(kmer_list, num_n)
            print(f'The length of grouped odd palindromes is {len(grouped_mers)}')
            merged_counts = merge_counts(grouped_mers, kmer_counts)
            merged_exps = merge_counts(grouped_mers, expec_dict)
            return compute_stats(merged_counts, merged_exps, len_genome, max_e)
        prefix, sufix, center = kmer_counts[kmer[:-1]], kmer_counts[kmer[1:]], kmer_counts[kmer[1:-1]]
        if center == 0:
            expected = 0
        else:
            expected = int((prefix * sufix) / center)
        observed = kmer_counts[kmer]
        sigma = math.sqrt(expected * (1 - expected/(len_genome-k+1)))
        if sigma == 0.0:
            z_score = 0.0
        else:
            z_score = ((observed - expected) / sigma)
        # pvalue for all kmers/palindromes under represented
        p_value_under = (math.erfc(-z_score / math.sqrt(2)) / 2)
        # pvalue for all kmers/palindromes over represented
        p_value_over = (math.erfc(z_score / math.sqrt(2)) / 2)
        # evalue for all kmers/palindromes under represented
        e_value_under = (n * p_value_under)
        # evalue for all kmers/palindromes over represented
        e_value_over = (n * p_value_over)
        if e_value_under <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_under, e_value_under])
        elif e_value_over <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_over, e_value_over])
    return results

In [81]:
dt = stats_kmer(p, cv, seq_len, 0.01, 1, all_kmers=False)

KeyError: 'AAAAAAAATTTTTTT'

In [238]:
pd.DataFrame(dt)

,0,1,2,3,4,5
0,AAATTT,2904,3380,-8.195020,1.252758e-16,8.017653e-15
1,AAGCTT,359,651,-11.446418,1.225317e-30,7.842028e-29
2,AATATT,2035,2409,-7.624989,1.220276e-14,7.809767e-13
3,AGCGCT,47,105,-5.660383,7.551785e-09,4.833143e-07
4,ATATAT,425,565,-5.890754,1.922192e-09,1.230203e-07
5,ATCGAT,387,480,-4.245407,1.090987e-05,6.982317e-04
6,ATGCAT,255,477,-10.166007,1.406011e-24,8.998468e-23
7,CAATTG,674,920,-8.112419,2.481090e-16,1.587898e-14
8,CACGTG,236,318,-4.598731,2.125359e-06,1.360230e-04
9,CAGCTG,310,411,-4.982522,3.138041e-07,2.008346e-05


In [233]:
all_k = stats_kmer(p_list, cv, seq_len, 0.01, 1, all_kmers=True)

In [235]:
pd.DataFrame(all_k)

,0,1,2,3,4,5
0,AAATTT,2904,3380,-8.195020,1.252758e-16,8.017653e-15
1,AAGCTT,359,651,-11.446418,1.225317e-30,7.842028e-29
2,AATATT,2035,2409,-7.624989,1.220276e-14,7.809767e-13
3,AGCGCT,47,105,-5.660383,7.551785e-09,4.833143e-07
4,ATATAT,425,565,-5.890754,1.922192e-09,1.230203e-07
5,ATCGAT,387,480,-4.245407,1.090987e-05,6.982317e-04
6,ATGCAT,255,477,-10.166007,1.406011e-24,8.998468e-23
7,CAATTG,674,920,-8.112419,2.481090e-16,1.587898e-14
8,CACGTG,236,318,-4.598731,2.125359e-06,1.360230e-04
9,CAGCTG,310,411,-4.982522,3.138041e-07,2.008346e-05


In [133]:
pal_k, grouped = stats_kmer(p_list, c, seq_len, 0.01, 1, all_kmers=False)

The length of expected values for odd palindromes is 64
The length of grouped odd palindromes is 16


In [139]:
pal_k

[['AANTT',
  23971,
  24525,
  -3.5615155327325043,
  0.00018436014512191956,
  0.002949762321950713],
 ['ACNGT',
  4340,
  5189,
  -11.802728273730553,
  1.8892923437861472e-32,
  3.0228677500578355e-31],
 ['AGNCT',
  4059,
  3744,
  5.153322449713376,
  1.2795568391579887e-07,
  2.047290942652782e-06],
 ['ATNAT',
  15254,
  16306,
  -8.275331735355868,
  6.40492379270908e-17,
  1.0247878068334528e-15],
 ['CCNGG',
  956,
  1723,
  -18.486609050357487,
  1.3234207158465212e-76,
  2.117473145354434e-75],
 ['CGNCG',
  2748,
  3307,
  -9.729431030600324,
  1.1292633735747245e-22,
  1.806821397719559e-21],
 ['CTNAG',
  3010,
  3640,
  -10.452548151669278,
  7.132019098113301e-26,
  1.1411230556981281e-24],
 ['GANTC',
  3140,
  4364,
  -18.55057849845495,
  4.0339507235519945e-77,
  6.454321157683191e-76],
 ['GCNGC',
  4842,
  5908,
  -13.891178265467623,
  3.58234627114441e-44,
  5.731754033831056e-43],
 ['GGNCC',
  793,
  1015,
  -6.970122603955302,
  1.5833244780001056e-12,
  2.533319164

In [151]:
all_kmers = False
if not all_kmers:
    print('hi')

hi


In [143]:
c['AAATT'], c['AACTT'], c['AAGTT'], c['AATTT'], sum([8840, 3155, 3155, 8821])

(8840, 3155, 3155, 8821, 23971)

In [145]:
e['AAATT'], e['AACTT'], e['AAGTT'], e['AATTT'], sum([8737, 3493, 3485, 8810])

(8737, 3493, 3485, 8810, 24525)

In [134]:
grouped

defaultdict(None,
            {'AANTT': ['AAATT', 'AACTT', 'AAGTT', 'AATTT'],
             'ACNGT': ['ACAGT', 'ACCGT', 'ACGGT', 'ACTGT'],
             'AGNCT': ['AGACT', 'AGCCT', 'AGGCT', 'AGTCT'],
             'ATNAT': ['ATAAT', 'ATCAT', 'ATGAT', 'ATTAT'],
             'CANTG': ['CAATG', 'CACTG', 'CAGTG', 'CATTG'],
             'CCNGG': ['CCAGG', 'CCCGG', 'CCGGG', 'CCTGG'],
             'CGNCG': ['CGACG', 'CGCCG', 'CGGCG', 'CGTCG'],
             'CTNAG': ['CTAAG', 'CTCAG', 'CTGAG', 'CTTAG'],
             'GANTC': ['GAATC', 'GACTC', 'GAGTC', 'GATTC'],
             'GCNGC': ['GCAGC', 'GCCGC', 'GCGGC', 'GCTGC'],
             'GGNCC': ['GGACC', 'GGCCC', 'GGGCC', 'GGTCC'],
             'GTNAC': ['GTAAC', 'GTCAC', 'GTGAC', 'GTTAC'],
             'TANTA': ['TAATA', 'TACTA', 'TAGTA', 'TATTA'],
             'TCNGA': ['TCAGA', 'TCCGA', 'TCGGA', 'TCTGA'],
             'TGNCA': ['TGACA', 'TGCCA', 'TGGCA', 'TGTCA'],
             'TTNAA': ['TTAAA', 'TTCAA', 'TTGAA', 'TTTAA']})

In [114]:
compute_stats(cm, em, seq_len, 0.01)

[['AANTT',
  23971,
  24525,
  -3.5615155327325043,
  0.00018436014512191956,
  0.002949762321950713],
 ['ACNGT',
  4340,
  5189,
  -11.802728273730553,
  1.8892923437861472e-32,
  3.0228677500578355e-31],
 ['AGNCT',
  4059,
  3744,
  5.153322449713376,
  1.2795568391579887e-07,
  2.047290942652782e-06],
 ['ATNAT',
  15254,
  16306,
  -8.275331735355868,
  6.40492379270908e-17,
  1.0247878068334528e-15],
 ['CCNGG',
  956,
  1723,
  -18.486609050357487,
  1.3234207158465212e-76,
  2.117473145354434e-75],
 ['CGNCG',
  2748,
  3307,
  -9.729431030600324,
  1.1292633735747245e-22,
  1.806821397719559e-21],
 ['CTNAG',
  3010,
  3640,
  -10.452548151669278,
  7.132019098113301e-26,
  1.1411230556981281e-24],
 ['GANTC',
  3140,
  4364,
  -18.55057849845495,
  4.0339507235519945e-77,
  6.454321157683191e-76],
 ['GCNGC',
  4842,
  5908,
  -13.891178265467623,
  3.58234627114441e-44,
  5.731754033831056e-43],
 ['GGNCC',
  793,
  1015,
  -6.970122603955302,
  1.5833244780001056e-12,
  2.533319164

In [266]:
fasta_d =  {'Acidiphilium': ['Data/bacteria_splitted/Acidiphilium/chromosomes/NC_009484.1_Acidiphilium_cryptum_JF_5_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidiphilium/chromosomes/NC_015186.1_Acidiphilium_multivorum_AIU301_complete_genome.fna.gz'],
             'Acidipropionibacterium': ['Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP015970.1_Acidipropionibacterium_acidipropionici_strain_F3E8_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP025198.1_Acidipropionibacterium_virtanenii_strain_JS278_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP013126.1_Acidipropionibacterium_acidipropionici_strain_CGMCC_1.2230_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP014352.1_Acidipropionibacterium_acidipropionici_strain_ATCC_55737_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP040635.1_Acidipropionibacterium_jensenii_strain_FAM_19038_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP019400.1_Acidipropionibacterium_acidipropionici_strain_WSH1105_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_LR134473.1_Acidipropionibacterium_jensenii_strain_NCTC13652_chromosome_1.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NC_019395.1_Acidipropionibacterium_acidipropionici_ATCC_4875_complete_sequence.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP025570.1_Acidipropionibacterium_jensenii_strain_JS280_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP031057.1_Acidipropionibacterium_acidipropionici_strain_WGS7_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP025571.1_Acidipropionibacterium_jensenii_strain_JS279_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidipropionibacterium/chromosomes/NZ_CP040634.1_Acidipropionibacterium_acidipropionici_strain_FAM_19036_chromosome_complete_genome.fna.gz'],
             'Acidisarcina': ['Data/bacteria_splitted/Acidisarcina/chromosomes/NZ_CP030840.1_Acidisarcina_polymorpha_strain_SBC82_chromosome_complete_genome.fna.gz'],
             'Acidithiobacillus': ['Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_LT841305.1_Acidithiobacillus_ferrivorans_isolate_PRJEB5721_chromosome_AFERRI.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NC_011761.1_Acidithiobacillus_ferrooxidans_ATCC_23270_complete_sequence.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_CP005986.1_Acidithiobacillus_caldus_ATCC_51756_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_CP044411.1_Acidithiobacillus_sp._AMD_consortium_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_CP043926.1_Acidithiobacillus_caldus_strain_MTH_04_chromosome.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NC_011206.1_Acidithiobacillus_ferrooxidans_ATCC_53993_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NC_015850.1_Acidithiobacillus_caldus_SM_1_complete_sequence.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_AP018795.1_Acidithiobacillus_ferridurans_strain_JCM_18981.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_CP026328.2_Acidithiobacillus_caldus_strain_MTH_04_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_CP040511.1_Acidithiobacillus_ferrooxidans_strain_YNTRS_40_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NZ_CP045571.1_Acidithiobacillus_thiooxidans_ATCC_19377_chromosome_complete_genome.fna.gz',
              'Data/bacteria_splitted/Acidithiobacillus/chromosomes/NC_015942.1_Acidithiobacillus_ferrivorans_SS3_complete_sequence.fna.gz']}

In [260]:
def get_lengths(fasta_dict):
    """Returns a dictionary like object with filenames as keys and
    sequence lengths as values.
    Input:
        fasta_dict - a dictionary like obj where the keys are the genera
        names and values are a list like object with the full path of the
        fasta files.
        ex = 
        {'Acidiphilium': ['Data/bacteria_splitted/Acidiphilium/chromosomes/NC_009484.1_Acidiphilium_cryptum_JF_5_complete_genome.fna.gz',
        'Data/bacteria_splitted/Acidiphilium/chromosomes/NC_015186.1_Acidiphilium_multivorum_AIU301_complete_genome.fna.gz']}
    Output:
        seq_len - a dictionary like obj where the keys are the filenames and
        the balues are integer representing the sequence lengths.
        defaultdict(int,
            {'NC_009484.1_Acidiphilium_cryptum_JF_5_complete_genome.fna': 3389227,
             'NC_015186.1_Acidiphilium_multivorum_AIU301_complete_genome.fna': 3749411})
    
    """
    seq_len = defaultdict(int)
    for name, filenames in fasta_dict.items():
        for path in filenames:
            filename = path.split('/')[-1]
            for n, seq in parse_fasta(path):
                seq_len[filename] = seq_len.get(name, 0) + len(seq)
    return seq_len           

In [267]:
get_lengths(fasta_d)

defaultdict(int,
            {'NC_009484.1_Acidiphilium_cryptum_JF_5_complete_genome.fna.gz': 3389227,
             'NC_015186.1_Acidiphilium_multivorum_AIU301_complete_genome.fna.gz': 3749411,
             'NZ_CP015970.1_Acidipropionibacterium_acidipropionici_strain_F3E8_chromosome_complete_genome.fna.gz': 3631890,
             'NZ_CP025198.1_Acidipropionibacterium_virtanenii_strain_JS278_chromosome_complete_genome.fna.gz': 3432872,
             'NZ_CP013126.1_Acidipropionibacterium_acidipropionici_strain_CGMCC_1.2230_chromosome_complete_genome.fna.gz': 3651382,
             'NZ_CP014352.1_Acidipropionibacterium_acidipropionici_strain_ATCC_55737_chromosome_complete_genome.fna.gz': 3704826,
             'NZ_CP040635.1_Acidipropionibacterium_jensenii_strain_FAM_19038_chromosome_complete_genome.fna.gz': 2999408,
             'NZ_CP019400.1_Acidipropionibacterium_acidipropionici_strain_WSH1105_chromosome_complete_genome.fna.gz': 3645455,
             'NZ_LR134473.1_Acidipropionibacterium_

In [258]:
for name, lis in fasta_dict.items():
    for e in lis:
        for n, seq in parse_fasta(e):
            print(len(seq))

3389227
3749411
3631890
3432872
3651382
3704826
2999408
3645455
3180547
3656170
3044937
3615145
3032477
3622795
7112011
3409655
2982397
2777717
2933811
2876631
2885038
2932225
2921399
2726366
3209933
3415726
3207552
6271114
4127356
2443540
4077222
4574859
3986972
5515908
3152246
4122625
5482170
5628961
4098001
5349924
4846466
4206721
5593946
5751886
5455909
5551793
5045878
4448856
3796573
5574669
5196935
4077847
5352772
4679990
5563503
5618386
4082498
3846621
3843574
3943032
4065905
4075289
4220749
3973763
2988937
4068899
4172609
4011552
3353930
4018724
3749210
4648418
3980182
3800835
4090120
3005353
3378307
3307484
3982722
2973544
3088433
3934099
3963235
4177548
4021920
3966912
3998399
4110074
4039997
3798644
3963137
4052350
3218664
4031418
3980027
3550511
4138791
3983069
3943485
3983848
2876784
4087343
3824973
3178335
3936362
3984166


KeyboardInterrupt: 

In [249]:
get_lengths(fasta_dict)

['Data/bacteria_splitted/Acidiphilium/chromosomes/NC_009484.1_Acidiphilium_cryptum_JF_5_complete_genome.fna.gz', 'Data/bacteria_splitted/Acidiphilium/chromosomes/NC_015186.1_Acidiphilium_multivorum_AIU301_complete_genome.fna.gz']
['NC_009484.1_Acidiphilium_cryptum_JF_5_complete_genome.fna', 'NC_015186.1_Acidiphilium_multivorum_AIU301_complete_genome.fna']


defaultdict(int,
            {'NC_009484.1_Acidiphilium_cryptum_JF_5_complete_genome.fna': 3389227,
             'NC_015186.1_Acidiphilium_multivorum_AIU301_complete_genome.fna': 3749411})

In [33]:
def get_kmers_from_sequence(sequence, alphabet, min_k, max_k):
    """
    Generate all DNA k-mers over the entirety of a sequence.
    Inputs:
    sequence - string where all kmers will be checked
    min_k: minimum DNA kmer length (int)
    max_k: maximum DNA kmer length (int)
    Output:
    yields all DNA kmers (str) of length min_k to max_k
    """
    limits = range(min_k, max_k + 1)
    for i in range(0, len(sequence) - max_k + 1):
        for j in limits:
            kmer = sequence[i:i+j]
            if set(kmer).issubset(alphabet):
                yield sequence[i:i+j]

In [194]:
%timeit Counter(get_kmers_from_sequence(sequence, 'ACGT', 1, 6))

7.89 s ± 2.36 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [34]:
def get_palindromes(alphabet, min_k, max_k):
    """Generates all DNA palindromes over the range from min_k to max_k.
    Inputs:
    min_k - minimum palindrome length (int)
    max_k - maximum palindrome length (int)
    Output:
    yields all possible DNA palindromes (str) of length min_k to max_k.
    Some definitions:
    A palindrome is defined as a sequence which is equal to its reverse-complement.
    Note: for odd length palindromes, the middle base does not need to be the same
    in the reverse-complement.
    Ex.: AAT is a legal palindrome even though its reverse-complement is ATT
    """
    for k in range(min_k, (max_k + 1)):
        for mer in itertools.product(alphabet, repeat=int(k / 2)):
            kmer = ''.join(mer)
            # even pal
            if k % 2 == 0:
                pal = kmer + get_reverse_complement(kmer)
                yield pal
            else:
                for base in alphabet:  # odd pal
                    pal = kmer + base + get_reverse_complement(kmer)
                    yield pal

In [69]:
def count_mers(sequence, mer_list, alphabet, kmin, kmax):
    """Returns a dictionary like object with kmers as keys and counts
    as values.
    Inputs:
        sequence - a sequence as a string
        alphabet - a sequence as string with all 
                   the allowed charcaters in the kmer
        kmin - integer represent the minimum kmerlength 
        kmax - integer represent the maximun kmer length 
    kmer = s substring of the sequence with length kmin<k<kmax)
    """
    print(f'The number of all possible kmers with length between {kmin} and {kmax} is {len(mer_list)}.\n')
    kmer_from_seq = list(get_kmers_from_sequence(sequence, alphabet, kmin, kmax))
    c = 0
    counts = defaultdict(int)
    for kmer in kmer_list:
        counts[kmer] = counts.get(kmer, 0)
    for kmer in kmer_from_seq:
        if kmer not in counts:
            counts[kmer] = 1
        else:
            counts[kmer] += 1
        c += 1
    print(f'The number of kmer counted in the sequence was {c}\n')
    return counts

In [68]:
def count_kmers(sequence, alphabet, min_k, max_k):
    alphabet = set(alphabet)
    counts = defaultdict(int)
    for kmer in get_kmers_from_sequence(sequence, alphabet, min_k, max_k):
        if set(kmer).issubset(alphabet):
            counts[kmer] = counts.get(kmer, 0) + 1
    return counts

In [ ]:
def get_all_dna_palindromes(alphabet, kmin, kmax):
    """A generator that yields all palindromic strings in a given length range, 
    including odd palindromes.
    Input:
        max_length - An integer specifying the maximum length palindromes to 
                     be yielded.
        min_length - An integer specifying the minimum length palindromes to
                     be yielded, if not specified defaults to 2.
    Output:
        palindrome - palindromic string length within the specified range.
                     An even length palindrome is defined as some sequence
                     ABCC'B'A' (C'B'A' = reverse complement of ABC).
                     We also allow for odd palindromes which are of the form:
                     ABCXC'B'A' where X is one of the two wildcards specifying 
                     charcters W for A or T, and S for C or G.
    
    This function is a generator that yields all possible palindromics in a given 
    length range. It allow for odd length palindromes, that presents a wild card
    charcter in the center of the palindrome. 
    I.E. we allow the following as valid palindromes:
      sequence:            5' AATATT   ...   AATCATT   ...  AATAATT 3'
      Reverse complement:  3' TTATAA   ...   AATGATT   ...  TTATTAA 5'
    
      yielded as:             AATATT         AATSATT        AATWATT
    """
    #Negative lengths make no sense in this context.
    if kmin < 0 or kmax < 0 or isinstance(kmin, float) or isinstance(kmax, float):
        print("ERROR: kmin or kmax cannot have negative values or be floats.",file=sys.stderr)
        sys.exit(1)
    #Each palindrome consists of length/2 bases, followed by its reverse complement
    #if even, or followed by one of two possible base pairs, and then the reverse
    #complement if odd, hence we generate each possible combination of "ACGT" for
    #length/2, and use this to construct palindromes.
    for k in range(kmin, kmax + 1):
        for kmer in itertools.product(alphabet,repeat=k//2):
            #Need to use join as product returns a tuple, not a string.
            kmer=''.join(kmer)
            if k % 2 == 0:
                palindrome = kmer + get_reverse_complement(kmer)
                yield palindrome
            else:
                for base in "WS":
                    palindrome = kmer + base + get_reverse_complement(kmer)
                    yield palindrome

In [16]:
len(list(get_all_dna_palindromes('ACGT', 5, 5)))*2

64

In [67]:
def subpalindrome_count(palindrome, kmer_counts):
    """Given a substring of a palindrome and a counts as a dictionary of kmer counts,
    it returns the count for substrings of even palindromes, or of odd length palindromes 
    containing a 'W' or 'S' degenerate nucleotide.
    Inputs:
        palindrome  - string palindromic, or substring.
        kmer_counts - a dictionary like object containing counts of kmer occurences 
                      of the same length as 'palindrome'.
    Output:
        count       - a integer specifying the count of occurences of 'palindrome' in 
                      the input data. For a substring of an even palindrome with
                      no wilde cards characters ('W' or 'S'), 'count' is simply 
                      kmer_counts value for tha palindrome. For a substring of an 
                      odd palindrome (with 'W' or 'S'), then the counts are merged
                      i.e, count = kmer_counts["AACTT"] + kmer_counts["AAGTT"].
                      If 'palindrome'have no values in kmers_counts it return 0.
    
    WARNING: This function is not intended for producing all possible interpretations
    of a substring containing degenerate nucleotide symbols. It's purpose is only to 
    simplify getting palindrome counts from kmer data, in which case substrings of odd 
    length palindromes may contain at most one 'W' or 'S' degenerate nucleotide.
    """
    #resolved_palindromes will store the resolved interpretations of 'palindrome', that is
    #both possibilities of a 'W' or 'S' character if present, or just 'palindrome' if no
    #'W' or 'S' is present. We can then iterate over resolved_palindromes to get the 
    #total count.
    resolved_palindromes = []
    count = 0
    if 'S' in palindrome:
        #'S' stands for 'G' or 'C', so generate both of these possbilities.
        resolved_palindromes.append(str.replace(palindrome,'S','C'))
        resolved_palindromes.append(str.replace(palindrome,'S','G'))
    elif 'W' in palindrome:
        #'W' stands for 'A' or 'T' so generate both possible interpretations.
        resolved_palindromes.append(str.replace(palindrome,'W','A'))
        resolved_palindromes.append(str.replace(palindrome,'W','T'))
    else: 
        return kmer_counts[palindrome]
    
    for resolved_palindrome in resolved_palindromes:
        count += kmer_counts[resolved_palindrome]
    return count

In [18]:
com = [''.join(kmer) for kmer in itertools.product('WS',repeat=3)]
com

['WWW', 'WWS', 'WSW', 'WSS', 'SWW', 'SWS', 'SSW', 'SSS']

In [66]:
def get_expected_kmer_values(mer_list, kmer_counts):
    expected = defaultdict(int)
    for kmer in mer_list:
        expected[kmer] = expected.get(kmer, 0)
        prefix, sufix, center = kmer_counts[kmer[:-1]], kmer_counts[kmer[1:]], kmer_counts[kmer[1:-1]]
        if center == 0:
            expected[kmer] = 0
        else:
            expected[kmer] += int((prefix * sufix) / center)
    return expected

In [65]:
def insert_wild_cards(word,num_wild_card=1):
    word = word.upper()
    mononuc = {'A': 'W', 
               'C': 'S', 
               'G': 'S', 
               'T': 'W'}
    wild_cards_dimers = {'AA': 'WW',  'AC': 'WS','AG': 'WS',
                         'AT': 'WW', 'CA': 'SW', 'CC': 'SS',
                         'CG': 'SS', 'CT': 'SW', 'GA': 'SW',
                         'GC': 'SS', 'GG': 'SS', 'GT': 'SW',
                         'TA': 'WW', 'TC': 'WS', 'TG': 'WS',
                         'TT': 'WW'}
    wild_cards_trimers = {'AAA': 'WWW', 'AAC': 'WWS', 'AAG': 'WWS',
                          'AAT': 'WWW', 'ACA': 'WSW', 'ACC': 'WSS',
                          'ACG': 'WSS', 'ACT': 'WSW', 'AGA': 'WSW',
                          'AGC': 'WSS', 'AGG': 'WSS', 'AGT': 'WSW',
                          'ATA': 'WWW', 'ATC': 'WWS', 'ATG': 'WWS',
                          'ATT': 'WWW', 'CAA': 'SWW', 'CAC': 'SWS',
                          'CAG': 'SWS', 'CAT': 'SWW', 'CCA': 'SSW',
                          'CCC': 'SSS', 'CCG': 'SSS', 'CCT': 'SSW',
                          'CGA': 'SSW', 'CGC': 'SSS', 'CGG': 'SSS',
                          'CGT': 'SSW', 'CTA': 'SWW', 'CTC': 'SWS',
                          'CTG': 'SWS', 'CTT': 'SWW', 'GAA': 'SWW',
                          'GAC': 'SWS', 'GAG': 'SWS', 'GAT': 'SWW',
                          'GCA': 'SSW', 'GCC': 'SSS', 'GCG': 'SSS',
                          'GCT': 'SSW', 'GGA': 'SSW', 'GGC': 'SSS',
                          'GGG': 'SSS', 'GGT': 'SSW', 'GTA': 'SWW',
                          'GTC': 'SWS', 'GTG': 'SWS', 'GTT': 'SWW',
                          'TAA': 'WWW', 'TAC': 'WWS', 'TAG': 'WWS',
                          'TAT': 'WWW', 'TCA': 'WSW', 'TCC': 'WSS',
                          'TCG': 'WSS', 'TCT': 'WSW', 'TGA': 'WSW',
                          'TGC': 'WSS', 'TGG': 'WSS', 'TGT': 'WSW',
                          'TTA': 'WWW', 'TTC': 'WWS', 'TTG': 'WWS',
                          'TTT': 'WWW'}
    mid = len(word)//2
    # all odd words can receive 1 wild card char
    if num_wild_card == 1 and word[:mid] == get_reverse_complement(word[mid+1:]) and len(word) % 2 != 0:
            return word[:mid] + mononuc[word[mid]] + word[mid+1:], word
    # the even words can receive two wild card chars
    if num_wild_card == 2 and word[:mid-1] == get_reverse_complement(word[mid+1:]) and len(word) % 2 == 0:
            return  word[:mid-1] + wild_cards_dimers[word[mid-1:mid+1]] + word[mid+1:], word
            # only odd words can return a word with 3 chars wild cards
    if num_wild_card == 3 and word[:mid-1] == get_reverse_complement( word[mid+2:]) and len(word) % 2 != 0 and\
    len(word) >= 5:
            return word[:mid-1] + wild_cards_trimers[word[mid-1:mid+2]] + word[mid+2:], word
    else:
        return word

In [110]:
insert_n('AAATT', num_n=1), insert_n('AACATT', num_n=2), insert_n('AAAATTT', num_n=3)

(('AANTT', 'AAATT'), ('AANNTT', 'AACATT'), ('AANNNTT', 'AAAATTT'))

In [106]:
c = 'AAAATTT'
p = len(c)//2
c[p], c[:p-1], c[p+2:]

('A', 'AA', 'TT')

In [64]:
def get_wild_cards_counts(kmer_list, num_wild_card, kmer_counts):
    wild_card_counts = defaultdict(int)
    wild_cards_tuples = [insert_wild_cards(kmer, num_wild_card=num_wild_card) for kmer in kmer_list]
    set_wild_cards = set([d[0] for d in wild_cards_tuples])
    final_counts = defaultdict(int)
    for kmer in set_wild_cards:
        final_counts[kmer] = final_counts.get(kmer, 0)
    for tup in wild_cards_tuples:
        wild_card_counts[tup] = wild_card_counts.get(tup, 0) + kmer_counts[tup[1]]

    return final_counts

In [27]:
pal_list = list(get_palindromes('ACGT', 5, 5))

In [25]:
kmer_list = list(get_all_possible_kmers('ACGT', 5, 5))
#kmer_from_seq = list(get_kmers_from_sequence(sequence, 'ACGT', 2, 6))

In [227]:
kmer_list[5]

'AAACC'

In [28]:
counts_kmers = count_kmers(sequence, 'ACGT', 1, 6)

In [35]:
counts = count_mers(sequence, kmer_list, 'ACGT', 1, 6)

The number of all possible kmers with length between 1 and 6 is 1024.

The number of kmer counted in the sequence was 5460



In [128]:
exp = get_expected_kmer_values(kmer_list, counts)

In [63]:
def count_ambiguous_bases(sequence):
    sequence = sequence.upper()
    amb = ['N', 'R', 'Y', 'W', 'S', 'K', 'M']
    return sum({base: sequence.count(base) for base in amb}.values())

def base_composition(sequence):
    """function to work out the 'C,G,A,T' content of the genome/sequence
    in question. The function takes a string as input"""
    s = sequence.upper()
    length = (len(s) - count_ambiguous_bases(s))
    bc= {'A': round(s.count('A') / length, 4),
         'T': round(s.count('T') / length, 4),
         'G': round(s.count('G') / length, 4),
         'C': round(s.count('C') / length, 4)}
    return bc

In [237]:
b_comp = base_composition(sequence)

In [238]:
b_comp

{'A': 0.3102, 'T': 0.3083, 'G': 0.1899, 'C': 0.1916}

In [ ]:
counts['AAACC']

In [62]:
def kmer_counts_from_dict(kmer_list, kmer_counts):
    cnt_km = defaultdict(int)
    # iterate through a list of kmers
    for kmer in kmer_list:
        # gets the kmer count from a dictionary of kmer:counts
        cnt_km[kmer] = cnt_km.get(kmer, 0) + counts[kmer]
    return cnt_km

In [245]:
pal_counts = kmer_counts_from_dict(pal_list, counts)

In [253]:
{k:cnt for k,cnt in b_comp.items() f}

{'A': 0.3102, 'T': 0.3083, 'G': 0.1899, 'C': 0.1916}

In [280]:
def zero_order_markov(kmer_counts, kmer, base_freq):
    kmer_freq = kmer_counts[kmer] / sum(kmer_counts.values())
    bases = [(base, base_freq[base]) for base in kmer]
    p = 1
    for base in base_freq:
        p *= base_freq[base]
    print(p, kmer_freq)
           

In [281]:
zero_order_markov(pal_counts, 'ACGGT', b_comp)

0.0034796518025544004 0.00985888427507958


In [282]:
0.00985888427507958/0.0034796518025544004

2.833296213098738

In [37]:
g = defaultdict(int)
wdk = [insert_wild_cards(pal,num_wild_card=1) for pal in pal_list]
set_wdk = set([d[0] for d in wdk])

for data in wdk:
    g[data] = g.get(data, 0) + counts[data[1]]

In [38]:
merged_couts = defaultdict(int)
for k in set_wdk:
    cnt = subpalindrome_count(k,  counts)
    merged_couts[k] += cnt

In [40]:
set_wdk = set([d[0] for d in wdk])
set_wdk, len(set_wdk)

({'AASTT',
  'AAWTT',
  'ACSGT',
  'ACWGT',
  'AGSCT',
  'AGWCT',
  'ATSAT',
  'ATWAT',
  'CASTG',
  'CAWTG',
  'CCSGG',
  'CCWGG',
  'CGSCG',
  'CGWCG',
  'CTSAG',
  'CTWAG',
  'GASTC',
  'GAWTC',
  'GCSGC',
  'GCWGC',
  'GGSCC',
  'GGWCC',
  'GTSAC',
  'GTWAC',
  'TASTA',
  'TAWTA',
  'TCSGA',
  'TCWGA',
  'TGSCA',
  'TGWCA',
  'TTSAA',
  'TTWAA'},
 32)

In [45]:
merged_couts

defaultdict(int,
            {'CAWTG': 6145,
             'GCWGC': 2881,
             'CASTG': 2128,
             'TGWCA': 2232,
             'CTSAG': 1062,
             'GASTC': 579,
             'TGSCA': 5715,
             'CTWAG': 1948,
             'CGWCG': 1160,
             'CCWGG': 698,
             'CCSGG': 258,
             'GGWCC': 274,
             'GGSCC': 519,
             'AAWTT': 17661,
             'ATSAT': 5852,
             'AASTT': 6310,
             'TTSAA': 9468,
             'ATWAT': 9402,
             'TAWTA': 7703,
             'TTWAA': 13257,
             'AGSCT': 2860,
             'GCSGC': 1961,
             'GTWAC': 2549,
             'AGWCT': 1199,
             'TASTA': 2089,
             'CGSCG': 1588,
             'TCSGA': 1125,
             'GTSAC': 1481,
             'GAWTC': 2561,
             'ACWGT': 1982,
             'TCWGA': 2684,
             'ACSGT': 2358})

In [46]:
counts['AAATT']

8840

In [61]:
def grouped_mers(data_dict):
    '''Function tha returns a dictionary like object that has
    the kmers as keys and counts as values.
    The kmers are grouped by similarities in the prefi and suffix.
    Ex., AAATT
         AACTT
         AAGTT
         AATTT...
    '''
    grouped_kmer = defaultdict(list)
    for kmer, lst in data_dict.items():
        half = len(kmer)//2
        if len(kmer) % 2 != 0 and len(kmer) >= 3 and kmer[:half] == get_reverse_complement(kmer[half+1:]):
            grouped_kmer[kmer] = grouped_kmer.get(kmer, []) + lst
        elif kmer[:half-1] == get_reverse_complement(kmer[half+1:]) and len(kmer) >= 3:
            grouped_kmer[kmer] = grouped_kmer.get(kmer, []) + lst
        elif kmer[:half-2] == get_reverse_complement(kmer[half+2:]) and len(kmer) >= 3:
            grouped_kmer[kmer] = grouped_kmer.get(kmer, []) + lst
        else:
            grouped_kmer[kmer] = grouped_kmer.get(kmer, []) + lst
    return grouped_kmer        

In [60]:
def compute_kmer_stats(kmer_list, counts, len_genome, max_e):
    """
    compute_stats computes the e-values for the supplied data.  
    Pre-conditions:
    'kmer_list' - a list of kmers (for which stats will be produced)
    'counts' - any dictionary-type with k-mers as keys (min_k - 2 <= k <= max_k,
    where min_k and max_k are the bounds on the k-mer lengths in 'kmer_list') 
    and counts as values.
    'N' - the total length of the sequence(s) read to produce 'counts'.
    'max_e' - the upper bound on e-values reported.
    Post-conditions:
    Retunrs a list of lists ('results') where results[i] is of the form
    [k-mer, observed count, expected count, z-score, e-value]
    """
    
    results = []
    #number of tests, used to convert p-value to e-value.
    n = len(kmer_list)
    for kmer in kmer_list:
        k = len(kmer)
        prefix, sufix, center = counts[kmer[:-1]], counts[kmer[1:]], counts[kmer[1:-1]]
        if center == 0:
            expected = 0
        else:
            expected = int((prefix * sufix) / center)
        observed = counts[kmer]
        sigma = math.sqrt(expected * (1 - expected/(len_genome-k+1)))
        if sigma == 0.0:
            z_score = 0.0
        else:
            z_score = ((observed - expected) / sigma)
        # pvalue for all kmers/palindromes under represented
        p_value_under = (math.erfc(-z_score / math.sqrt(2)) / 2)
        # pvalue for all kmers/palindromes over represented
        p_value_over = (math.erfc(z_score / math.sqrt(2)) / 2)
        # evalue for all kmers/palindromes under represented
        e_value_under = (n * p_value_under)
        # evalue for all kmers/palindromes over represented
        e_value_over = (n * p_value_over)
        if e_value_under <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_under, e_value_under])
        elif e_value_over <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_over, e_value_over])
    return results

In [50]:
len(kmer_list) #, (len(set(kmer_from_seq)))

1024

In [51]:
len(pal_list)

64

In [139]:
def hamming_distance(s1, s2):
    """Return the Hamming distance between equal-length sequences"""
    if len(s1) != len(s2):
        raise ValueError("Undefined for sequences of unequal length")
    return sum(ch1 != ch2 for ch1, ch2 in zip(s1, s2))



def percent_id(s1, s2):
    """Return the percentage identity between two strings.
       Strings must be the same length or aligned to the same length for the
       underlying Hamming Distance calculation to work.
    """
    try:
        hd = hamming_distance(s1, s2)
    except ValueError:
        raise ValueError("Sequences must be the same length and/or aligned.")
    return round(float(((len(s1) - hd) * 100) / len(s1)), 2)

In [142]:
grouped_pals = get_grouped_palindromes(pal_list, 1)

In [143]:
grouped_pals

defaultdict(None,
            {'AANTT': ['AAATT', 'AACTT', 'AAGTT', 'AATTT'],
             'ACNGT': ['ACAGT', 'ACCGT', 'ACGGT', 'ACTGT'],
             'AGNCT': ['AGACT', 'AGCCT', 'AGGCT', 'AGTCT'],
             'ATNAT': ['ATAAT', 'ATCAT', 'ATGAT', 'ATTAT'],
             'CANTG': ['CAATG', 'CACTG', 'CAGTG', 'CATTG'],
             'CCNGG': ['CCAGG', 'CCCGG', 'CCGGG', 'CCTGG'],
             'CGNCG': ['CGACG', 'CGCCG', 'CGGCG', 'CGTCG'],
             'CTNAG': ['CTAAG', 'CTCAG', 'CTGAG', 'CTTAG'],
             'GANTC': ['GAATC', 'GACTC', 'GAGTC', 'GATTC'],
             'GCNGC': ['GCAGC', 'GCCGC', 'GCGGC', 'GCTGC'],
             'GGNCC': ['GGACC', 'GGCCC', 'GGGCC', 'GGTCC'],
             'GTNAC': ['GTAAC', 'GTCAC', 'GTGAC', 'GTTAC'],
             'TANTA': ['TAATA', 'TACTA', 'TAGTA', 'TATTA'],
             'TCNGA': ['TCAGA', 'TCCGA', 'TCGGA', 'TCTGA'],
             'TGNCA': ['TGACA', 'TGCCA', 'TGGCA', 'TGTCA'],
             'TTNAA': ['TTAAA', 'TTCAA', 'TTGAA', 'TTTAA']})

In [113]:
# grouped_pal = []
# for a, b in itertools.combinations(pal_list, 2):
#     if hamming_distance(a, b) == 1:
#         print(a, '>', b)
#         grouped_pal.append(b)

In [125]:
counts['ACAGT'], counts['ACCGT'], counts['ACGGT'], counts['ACTGT']

(1034, 1178, 1180, 948)

In [126]:
sum([8840, 3155, 3155, 8821]), sum([1034, 1178, 1180, 948])

(23971, 4340)

In [58]:
def merge_counts(grouped_pal_dict, kmer_counts):
    
    merged = defaultdict(int)
    # iterates through keys and values in the dict of grouped palindromes
    for g, pals in grouped_pal_dict.items():
        # iterates through the values lists (grouped kmers)
        for pal in pals:
            # sum up all the grouped palindromes counts
            merged[g] += kmer_counts[pal]
    return merged

In [171]:
merge_cnts = merge_counts(grouped_pals, counts)

In [172]:
merge_cnts

defaultdict(int,
            {'AANTT': 23971,
             'ACNGT': 4340,
             'AGNCT': 4059,
             'ATNAT': 15254,
             'CANTG': 8273,
             'CCNGG': 956,
             'CGNCG': 2748,
             'CTNAG': 3010,
             'GANTC': 3140,
             'GCNGC': 4842,
             'GGNCC': 793,
             'GTNAC': 4030,
             'TANTA': 9792,
             'TCNGA': 3809,
             'TGNCA': 7947,
             'TTNAA': 22725})

In [162]:
sum([counts['CCAGG'], counts['CCCGG'], counts['CCGGG'], counts['CCTGG']])

956

In [ ]:
{'AANTT': ['AAATT', 'AACTT', 'AAGTT', 'AATTT'],
             'ACNGT': ['ACAGT', 'ACCGT', 'ACGGT', 'ACTGT'],
             'AGNCT': ['AGACT', 'AGCCT', 'AGGCT', 'AGTCT'],
             'ATNAT': ['ATAAT', 'ATCAT', 'ATGAT', 'ATTAT'],
             'CANTG': ['CAATG', 'CACTG', 'CAGTG', 'CATTG'],
             'CCNGG': ['CCAGG', 'CCCGG', 'CCGGG', 'CCTGG'],
             'CGNCG': ['CGACG', 'CGCCG', 'CGGCG', 'CGTCG'],
             'CTNAG': ['CTAAG', 'CTCAG', 'CTGAG', 'CTTAG'],
             'GANTC': ['GAATC', 'GACTC', 'GAGTC', 'GATTC'],
             'GCNGC': ['GCAGC', 'GCCGC', 'GCGGC', 'GCTGC'],
             'GGNCC': ['GGACC', 'GGCCC', 'GGGCC', 'GGTCC'],
             'GTNAC': ['GTAAC', 'GTCAC', 'GTGAC', 'GTTAC'],
             'TANTA': ['TAATA', 'TACTA', 'TAGTA', 'TATTA'],
             'TCNGA': ['TCAGA', 'TCCGA', 'TCGGA', 'TCTGA'],
             'TGNCA': ['TGACA', 'TGCCA', 'TGGCA', 'TGTCA'],
             'TTNAA': ['TTAAA', 'TTCAA', 'TTGAA', 'TTTAA']})

In [164]:
sum([exp['TTAAA'], exp['TTCAA'], exp['TTGAA'], exp['TTTAA']])

24459

In [156]:
merge_expecs = merge_counts(grouped_pals, exp)

In [198]:
merge_cnts

defaultdict(int,
            {'AANTT': 23971,
             'ACNGT': 4340,
             'AGNCT': 4059,
             'ATNAT': 15254,
             'CANTG': 8273,
             'CCNGG': 956,
             'CGNCG': 2748,
             'CTNAG': 3010,
             'GANTC': 3140,
             'GCNGC': 4842,
             'GGNCC': 793,
             'GTNAC': 4030,
             'TANTA': 9792,
             'TCNGA': 3809,
             'TGNCA': 7947,
             'TTNAA': 22725})

In [213]:
def compute_stats_grouped_kmers(kmer_counts_merged, kmer_expec_merged, len_genome, max_e):
    """
    """
    
    results = []
    #number of tests, used to convert p-value to e-value.
    kmers = list(kmer_counts_merged.keys())
    n = len(kmers)
    for kmer in kmers:
        observed =  kmer_counts_merged[kmer]
        expected = kmer_expec_merged[kmer]
        sigma = math.sqrt(expected * (1 - expected/(len_genome - len(kmer) + 1)))
        if sigma == 0.0:
            z_score = 0.0
        else:
            z_score = ((kmer_counts_merged[kmer] - kmer_expec_merged[kmer]) / sigma)
        # pvalue for all kmers/palindromes under represented
        p_value_under = (math.erfc(-z_score / math.sqrt(2)) / 2)
        # pvalue for all kmers/palindromes over represented
        p_value_over = (math.erfc(z_score / math.sqrt(2)) / 2)
        # evalue for all kmers/palindromes under represented
        e_value_under = (n * p_value_under)
        # evalue for all kmers/palindromes over represented
        e_value_over = (n * p_value_over)
        if e_value_under <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_under, e_value_under])
        elif e_value_over <= max_e:
            results.append([kmer, observed, expected, z_score, p_value_over, e_value_over])
    return results

In [196]:
data = compute_stats(count_groups, expec_groups, seq_len, 0.01)

In [302]:
len(data)

13

In [227]:
def merged_stats(data, grouped_pals):
    df1 = pd.DataFrame(grouped_pals.items(), columns=['kmer', 'groups'])
    df = pd.DataFrame(data, columns=['kmer', 
                            'count', 
                            'expected', 
                            'z_score', 
                            'p_value', 
                            'e_value']).sort_values('z_score').reset_index(drop=True)
    df_final = pd.merge(df, df1)
    df_final['group_len'] = df_final['groups'].str.len()
    return df_final

In [228]:
merged_stats = merged_stats(data, grouped_pal)

In [313]:
sum(merged_stats['group_len'])

52

In [230]:
merged_stats

,kmer,count,expected,z_score,p_value,e_value,groups,group_len
0,GTNNAC,221,923,-23.112440,1.735712e-118,2.777139e-117,"[GTATAC, GTCGAC, GTGCAC, GTTAAC]",4
1,GANNTC,577,1310,-20.259271,1.471436e-91,2.354297e-90,"[GAATTC, GACGTC, GAGCTC, GATATC]",4
2,AANNTT,6145,7363,-14.223133,3.291895e-46,5.267033e-45,"[AAATTT, AACGTT, AAGCTT, AATATT]",4
3,GGNNCC,220,527,-13.375056,4.229504e-41,6.767206e-40,"[GGATCC, GGCGCC, GGGCCC, GGTACC]",4
4,CANNTG,1394,1986,-13.291312,1.299896e-40,2.079834e-39,"[CAATTG, CACGTG, CAGCTG, CATATG]",4
5,TANNTA,1818,2388,-11.671891,8.871754e-32,1.419481e-30,"[TAATTA, TACGTA, TAGCTA, TATATA]",4
6,TTNNAA,4607,5270,-9.146068,2.952184e-20,4.723494e-19,"[TTATAA, TTCGAA, TTGCAA, TTTAAA]",4
7,ATNNAT,3019,3553,-8.967381,1.518239e-19,2.429183e-18,"[ATATAT, ATCGAT, ATGCAT, ATTAAT]",4
8,TGNNCA,1230,1567,-8.516897,8.194238e-18,1.311078e-16,"[TGATCA, TGCGCA, TGGCCA, TGTACA]",4
9,CGNNCG,412,588,-7.259284,1.945724e-13,3.113159e-12,"[CGATCG, CGCGCG, CGGCCG, CGTACG]",4


In [206]:
df1 = pd.DataFrame(grouped_pal.items(), columns=['kmer', 'groups'])

In [207]:
df1

,kmer,groups
0,AANNTT,"[AAATTT, AACGTT, AAGCTT, AATATT]"
1,ACNNGT,"[ACATGT, ACCGGT, ACGCGT, ACTAGT]"
2,AGNNCT,"[AGATCT, AGCGCT, AGGCCT, AGTACT]"
3,ATNNAT,"[ATATAT, ATCGAT, ATGCAT, ATTAAT]"
4,CANNTG,"[CAATTG, CACGTG, CAGCTG, CATATG]"
5,CCNNGG,"[CCATGG, CCCGGG, CCGCGG, CCTAGG]"
6,CGNNCG,"[CGATCG, CGCGCG, CGGCCG, CGTACG]"
7,CTNNAG,"[CTATAG, CTCGAG, CTGCAG, CTTAAG]"
8,GANNTC,"[GAATTC, GACGTC, GAGCTC, GATATC]"
9,GCNNGC,"[GCATGC, GCCGGC, GCGCGC, GCTAGC]"


In [208]:
df = pd.DataFrame(data, columns=['kmer', 
                            'count', 
                            'expected', 
                            'z_score', 
                            'p_value', 
                            'e_value']).sort_values('z_score').reset_index(drop=True)

In [297]:
df.size

78

In [214]:
df_final = pd.merge(df, df1)

In [215]:
def get_grouped_data_frame(data_merged):
    df = pd.DataFrame(data_merged,
                      columns=['kmer',
                               'count', 
                               'expected', 
                               'z_score', 
                               'p_value', 
                               'e_value']).sort_values('z_score').reset_index(drop=True)
    return df

In [301]:
get_grouped_data_frame(data)

,kmer,count,expected,z_score,p_value,e_value
0,GTNNAC,221,923,-23.112440,1.735712e-118,2.777139e-117
1,GANNTC,577,1310,-20.259271,1.471436e-91,2.354297e-90
2,AANNTT,6145,7363,-14.223133,3.291895e-46,5.267033e-45
3,GGNNCC,220,527,-13.375056,4.229504e-41,6.767206e-40
4,CANNTG,1394,1986,-13.291312,1.299896e-40,2.079834e-39
5,TANNTA,1818,2388,-11.671891,8.871754e-32,1.419481e-30
6,TTNNAA,4607,5270,-9.146068,2.952184e-20,4.723494e-19
7,ATNNAT,3019,3553,-8.967381,1.518239e-19,2.429183e-18
8,TGNNCA,1230,1567,-8.516897,8.194238e-18,1.311078e-16
9,CGNNCG,412,588,-7.259284,1.945724e-13,3.113159e-12


In [300]:
len(get_grouped_data_frame(data))

13

In [139]:
exp['ACAGT'], exp['ACCGT'], exp['ACGGT'], exp['ACTGT']

(1122, 1469, 1460, 1138)

In [140]:
sum([1122, 1469, 1460, 1138])

5189

In [118]:
# counts_dict = defaultdict(int)

# for km, lst in wild_cards_dict.items():
#     for mer in lst:
#         counts_dict[km] = counts_dict.get(km, 0) + cnt_km[mer]

In [105]:
final_count = defaultdict(int)
for kmw in counts_dict:
    if kmw[:len(kmw)//2] == get_reverse_complement(kmw[len(kmw)//2+1:]) and kmw[len(kmw)//2] in 'WS':
        final_count[kmw] += counts_dict[kmw]

In [106]:
final_count

defaultdict(int,
            {'AAWTT': 17661,
             'AASTT': 6310,
             'ACWGT': 1982,
             'ACSGT': 2358,
             'AGWCT': 1199,
             'AGSCT': 2860,
             'ATWAT': 9402,
             'ATSAT': 5852,
             'CAWTG': 6145,
             'CASTG': 2128,
             'CCWGG': 698,
             'CCSGG': 258,
             'CGWCG': 1160,
             'CGSCG': 1588,
             'CTWAG': 1948,
             'CTSAG': 1062,
             'GAWTC': 2561,
             'GASTC': 579,
             'GCWGC': 2881,
             'GCSGC': 1961,
             'GGWCC': 274,
             'GGSCC': 519,
             'GTWAC': 2549,
             'GTSAC': 1481,
             'TAWTA': 7703,
             'TASTA': 2089,
             'TCWGA': 2684,
             'TCSGA': 1125,
             'TGWCA': 2232,
             'TGSCA': 5715,
             'TTWAA': 13257,
             'TTSAA': 9468})

In [30]:
operator.eq(counts_kmers, counts)

True

In [32]:
len(exp), len(counts)

(1024, 5460)

In [262]:
def clustering_mers(mer_list):
    mer_ar = np.asarray(mer_list)
    clusters = defaultdict(list)
    levenshtein_similarity = - 1 * np.array([[distance.levenshtein(w1, w2) for w1 in mer_ar] 
                                     for w2 in mer_ar])
    affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", 
                                                  damping=0.5, 
                                                  random_state=4)
    affprop.fit(levenshtein_similarity)
    for ci in np.unique(affprop.labels_):
        exemplar = mer_ar[affprop.cluster_centers_indices_[ci]]
        cluster = np.unique(mer_ar[np.nonzero(affprop.labels_ == ci)])
        clist = [mer for mer in cluster]
        for member in clist:
            if percent_id(exemplar, member) >= 75.0:
                clusters[exemplar] = clusters.get(exemplar, []) + [member]
    return clusters

In [263]:
clustered_pal = clustering_mers(p_list)

In [264]:
clustered_pal

defaultdict(list,
            {'ACGCGT': ['ACGCGT'],
             'AGCGCT': ['AGCGCT'],
             'CCCGGG': ['CCCGGG'],
             'CGGCCG': ['CGGCCG'],
             'GATATC': ['GATATC'],
             'GTATAC': ['GTATAC'],
             'TAATTA': ['TAATTA'],
             'TTTAAA': ['TTTAAA']})

In [247]:
merge = defaultdict(int)
for k, v in clustered_pal.items():
    for km in v:
        merge[k] += counts[km]
    

In [250]:
merge

defaultdict(int,
            {'AACTT': 23971,
             'ACAGT': 4340,
             'AGACT': 4059,
             'ATCAT': 15254,
             'CACTG': 8273,
             'CCGGG': 956,
             'CGACG': 2748,
             'CTAAG': 3010,
             'GAGTC': 3140,
             'GCCGC': 4842,
             'GGACC': 793,
             'GTCAC': 4030,
             'TAGTA': 9792,
             'TCGGA': 3809,
             'TGACA': 7947,
             'TTTAA': 22725})

In [252]:
clustered_pal

defaultdict(list,
            {'AACTT': ['AAATT', 'AACTT', 'AAGTT', 'AATTT'],
             'ACAGT': ['ACAGT', 'ACCGT', 'ACGGT', 'ACTGT'],
             'AGACT': ['AGACT', 'AGCCT', 'AGGCT', 'AGTCT'],
             'ATCAT': ['ATAAT', 'ATCAT', 'ATGAT', 'ATTAT'],
             'CACTG': ['CAATG', 'CACTG', 'CAGTG', 'CATTG'],
             'CCGGG': ['CCAGG', 'CCCGG', 'CCGGG', 'CCTGG'],
             'CGACG': ['CGACG', 'CGCCG', 'CGGCG', 'CGTCG'],
             'CTAAG': ['CTAAG', 'CTCAG', 'CTGAG', 'CTTAG'],
             'GAGTC': ['GAATC', 'GACTC', 'GAGTC', 'GATTC'],
             'GCCGC': ['GCAGC', 'GCCGC', 'GCGGC', 'GCTGC'],
             'GGACC': ['GGACC', 'GGCCC', 'GGGCC', 'GGTCC'],
             'GTCAC': ['GTAAC', 'GTCAC', 'GTGAC', 'GTTAC'],
             'TAGTA': ['TAATA', 'TACTA', 'TAGTA', 'TATTA'],
             'TCGGA': ['TCAGA', 'TCCGA', 'TCGGA', 'TCTGA'],
             'TGACA': ['TGACA', 'TGCCA', 'TGGCA', 'TGTCA'],
             'TTTAA': ['TTAAA', 'TTCAA', 'TTGAA', 'TTTAA']})

In [251]:
final = defaultdict(int)
for k, v in merge.items():
    nk = k[:len(k)//2] + 'N' + k[len(k)//2+1:]
    final[nk] = final.get(nk, 0) + merge[k]
final

defaultdict(int,
            {'AANTT': 23971,
             'ACNGT': 4340,
             'AGNCT': 4059,
             'ATNAT': 15254,
             'CANTG': 8273,
             'CCNGG': 956,
             'CGNCG': 2748,
             'CTNAG': 3010,
             'GANTC': 3140,
             'GCNGC': 4842,
             'GGNCC': 793,
             'GTNAC': 4030,
             'TANTA': 9792,
             'TCNGA': 3809,
             'TGNCA': 7947,
             'TTNAA': 22725})

In [55]:
def neighbors(pattern, ham_dist):
    nucleotides = ['A','C','G','T']
    if ham_dist == 0:
        return pattern
    if len(pattern) == 1:
        return nucleotides

    neighborhood = []
    suffix = pattern[1:]
    for neighbor in neighbors(suffix, ham_dist):
        if hamming_distance(suffix, neighbor) < ham_dist:
            neighborhood.extend(['A' + neighbor,'C' + neighbor,'G' + neighbor,'T' + neighbor])
        else:
            neighborhood.append(pattern[0] + neighbor)

    return neighborhood

In [221]:
neighbors("TTAAA", 1)

['TAAAA',
 'TCAAA',
 'TGAAA',
 'ATAAA',
 'CTAAA',
 'GTAAA',
 'TTAAA',
 'TTCAA',
 'TTGAA',
 'TTTAA',
 'TTACA',
 'TTAGA',
 'TTATA',
 'TTAAC',
 'TTAAG',
 'TTAAT']

In [54]:
def neighbors_at_exact_ham_dist(pattern, ham_dist):
    nucleotides = ['A','C','G','T']
    if ham_dist == 0:
        return pattern
    if len(pattern) == 1:
        return nucleotides

    neighborhood = []
    suffix = pattern[1:]
    for neighbor in neighbors(suffix, ham_dist):
        if hamming_distance(suffix, neighbor) == ham_dist:
            neighborhood.extend(['A' + neighbor,'C' + neighbor,'G' + neighbor,'T' + neighbor])
        else:
            neighborhood.append(pattern[0] + neighbor)

    return neighborhood

In [222]:
neighbors_at_exact_ham_dist('TTAAA', 1)

['AAAAA',
 'CAAAA',
 'GAAAA',
 'TAAAA',
 'ACAAA',
 'CCAAA',
 'GCAAA',
 'TCAAA',
 'AGAAA',
 'CGAAA',
 'GGAAA',
 'TGAAA',
 'TTAAA',
 'ATCAA',
 'CTCAA',
 'GTCAA',
 'TTCAA',
 'ATGAA',
 'CTGAA',
 'GTGAA',
 'TTGAA',
 'ATTAA',
 'CTTAA',
 'GTTAA',
 'TTTAA',
 'ATACA',
 'CTACA',
 'GTACA',
 'TTACA',
 'ATAGA',
 'CTAGA',
 'GTAGA',
 'TTAGA',
 'ATATA',
 'CTATA',
 'GTATA',
 'TTATA',
 'ATAAC',
 'CTAAC',
 'GTAAC',
 'TTAAC',
 'ATAAG',
 'CTAAG',
 'GTAAG',
 'TTAAG',
 'ATAAT',
 'CTAAT',
 'GTAAT',
 'TTAAT']

In [268]:
def comb(N,k):
    """Combinations of N things taken k at a time.
    Exact long integer is computed.  Based on function from scipy.

    Notes:
      - If k > N, N < 0, or k < 0, then a 0 is returned.
    """
    if (k > N) or (N < 0) or (k < 0):
        return 0
    val = 1
    for j in range(min(k, N-k)):
        val = (val*(N-j))//(j+1)
    return val

In [269]:
def prob(full_length, mismatches, p_match) :
    """ this function is used to work out the probability of a palindrome of
    length - 'length', with x amount of mismatches, based on the probability of
    getting a match (p_match)

    the combination function (comb) is used instead of scipy to work out the number
    of possible ways of getting that mismatch and length combination to work out
    the probability of that occuring at random

    probability is defined by:
    p = combinatons of mismatche (N choose K) * (probability of a match based on the
    codon bias ** number of matches) * (probability of a mismatch ** number of
    mismatches)

    (** to the power of)
    """
    halflength= full_length //2
    matches= halflength - mismatches
    combinations_of_mismatch = comb(halflength,mismatches)
    p_mismatch = 1 - p_match
    return combinations_of_mismatch * ((p_match ** matches) * (p_mismatch ** mismatches))

In [270]:
def base_composition_sequence_probability(sequence, base_compo):
    """this is a function to return the probability of a given DNA sequence
    occuring at random. The base composition of the given string is taken
    into consideration(bc). dna should be a Bio.Seq seq. The given DNA sequence is
    treated as a pallindrome
    for a given sequence we need to find if its opposite palindromic pair
    is a match of miss match"""
    #we only need to use half of the length of the sequence so...
    halflength = len(sequence)//2
    #call in function to return mismatches in the sequence
    mismatches = count_mismatches(sequence, halflength)
    #work out the number of matches in a sequence
    matches = halflength - mismatches
    #we need to make a list of all the matching and mismatching pairs
    #the number of ways of getting that num of mismatches in a given sequence
    combinations_of_mismatch = comb(halflength, mismatches)
    #update values of bc using function
    #bc=base_composition(dna)
    #print bc
    matching_bases = ['AT', 'TA', 'GC', 'CG']
    p_match = sum([base_compo[m[0]] * base_compo[m[1]] for m in matching_bases])
    p_mismatch = 1 - p_match
    #the formula to work out the probability of a given sequence occuring
    #randomly in a genome.
    probability = combinations_of_mismatch * ((p_match ** matches) * (p_mismatch ** mismatches))
    #print 'pmatch', p_match
    return probability, mismatches

In [272]:
bases = base_composition('AAATT')
base_composition_sequence_probability('AAATT', bases)

(0.2304, 0)

In [273]:
count_mismatches('AAATT', len('AAATT'))

1

In [274]:
def iteration_length(data, lower, higher, probability_threshold=None):
    """this is a funtion to define the length of SIR we are looking for,

    lower to higher is the range of lengths we are interested in and should be an
    int.

    data is a string.
    
    function should return sequences that only fullfill the limits defined by
    is_palindrome function, the lenth of this seq and the start position.
    """
    complement_trans_table = str.maketrans("ACGTacgt", "TGCAtgca")
    data_comp = data.translate(complement_trans_table)

    #call function to work out the codon bias to work out the probability
    #based on the codon bias of the whole genome...
    bc = base_composition(data)

    #define maches bases
    matching_bases= ['AT', 'TA', 'GC', 'CG']
    #probability of a match... work out based on codon bias (bc)
    p_match = 0
    for i in matching_bases:
        p_match += bc[i[0]] * bc[i[1]]

    assert p_match == sum([bc[m[0]] * bc[m[1]] for m in matching_bases])

    #Big speed boost, cache the probabilities - these don't need to look at
    #actual potential palindrome sequence.
    #create dictionary of possible probability using length and mismatches
    #as the indexes - look up mismatches and length, returns the probability
    prob_dict = {}
    for length in range(lower, higher):
        for mismatches in range(0, (length // 2) + 1) :
            #call prob function to work out the probability... put in dict
            prob_dict[(length, mismatches)] = prob(length, mismatches, p_match)

    #create an empty list to put the result in
    palindrome_list = []
    for length in range(lower, higher):
        #print "Searching for length %i" % length
        #defined mismatch threshold - - - - IMPORTANT!!!!
        half_length = length // 2
        mismatches_threshold = 0.5 * half_length
        half_length_list = range(half_length)
        for start in range(0, (len(data) - length + 1)):
            seq = data[start:start+length]
            seq_comp = data_comp[start:start+length]
            mismatches = 0
            for i in half_length_list :
                if seq[i] != seq_comp[-i-1] :
                    mismatches += 1
            if mismatches <= mismatches_threshold :
                #Only now look up the probability,
                probability = prob_dict[(length, mismatches)]

                #assert probability, missmatches == base_composition_sequence_probability(seq, bc)

                if probability <= probability_threshold :
                    #results into list, as a list of tuples
                    palindrome_list += [(length, start, probability, mismatches, seq)]
    return palindrome_list


In [276]:
iteration_length(sequence, 6, 15, 0.5)

[(6, 2, 0.15392608165292607, 1, 'TGGCAA'),
 (6, 5, 0.15392608165292607, 1, 'CAATTA'),
 (6, 7, 0.15392608165292607, 1, 'ATTAAA'),
 (6, 8, 0.15392608165292607, 1, 'TTAAAA'),
 (6, 10, 0.15392608165292607, 1, 'AAAATT'),
 (6, 11, 0.15392608165292607, 1, 'AAATTG'),
 (6, 16, 0.15392608165292607, 1, 'GGTATC'),
 (6, 19, 0.15392608165292607, 1, 'ATCAAT'),
 (6, 21, 0.15392608165292607, 1, 'CAATGG'),
 (6, 39, 0.018407899636691323, 0, 'CGGCCG'),
 (6, 49, 0.15392608165292607, 1, 'GTATTC'),
 (6, 56, 0.15392608165292607, 1, 'GTGCAG'),
 (6, 72, 0.15392608165292607, 1, 'TGATGA'),
 (6, 89, 0.15392608165292607, 1, 'TAGGTA'),
 (6, 94, 0.15392608165292607, 1, 'ATTAAC'),
 (6, 102, 0.15392608165292607, 1, 'CTTAAT'),
 (6, 106, 0.15392608165292607, 1, 'ATCGAC'),
 (6, 109, 0.15392608165292607, 1, 'GACGTT'),
 (6, 115, 0.15392608165292607, 1, 'GAATAC'),
 (6, 117, 0.15392608165292607, 1, 'ATACAT'),
 (6, 119, 0.15392608165292607, 1, 'ACATGG'),
 (6, 126, 0.15392608165292607, 1, 'TTATAT'),
 (6, 127, 0.1539260816529260

In [49]:
def count_mismatches(seq, half_len):
    """ When passed a str(Bio.Seq seq), representing a nucleotide sequence,
        treats it as a short inverted repeat, and returns the number of 
        mismatched compared to its reverse complement for half the length
        of the sequence
    """    
    complement_trans_table = str.maketrans("ACGTacgt", "TGCAtgca") 
    #assert half_len == len(seq)/2 #TODO - for testing, this is SLOW!
    #half_len = len(seq)/2
    #Want to use the python string's translate method to map A->T etc.
    second_half_seq_comp = seq[-half_len:].translate(complement_trans_table)
    mismatches = 0
    for i in range(half_len) :
        if seq[i] != second_half_seq_comp[-i-1] :
            mismatches += 1
    return mismatches

In [277]:
count_mismatches('GTTAAC', 3)

0

In [48]:
def get_closest_lev(cluster_center_words, my_word):
    closest_center = None
    smallest_distance = float('inf')
    for word in cluster_center_words:
        ld = levenshtein(word, my_word)
        if ld < smallest_distance:
            smallest_distance = ld
            closest_center = word
    return closest_center

def get_clusters(cluster_center_words, other_words):
    cluster_dict = {}
    for word in cluster_center_words:
        cluster_dict[word] = []
    for my_word in other_words:
        closest_center = get_closest_lev(cluster_center_words, my_word)
        cluster_dict[closest_center].append(my_word)
    return cluster_dict

In [37]:
csv_file = 'Results/kmer_stats/Haemophilus/chromosomes/palindromes/Haemophilus_palindromes_5.csv'

In [38]:
df = pd.read_csv(csv_file)
df

,kmer,count,expected,z_score,p_value,e_value
0,CCTGG,342,784,-15.789097,1.849189e-56,1.183481e-54
1,CCAGG,356,766,-14.817003,5.687327e-50,3.639889e-48
2,GAATC,1304,1839,-12.481916,4.685241e-36,2.998554e-34
3,GATTC,1257,1774,-12.280734,5.747504e-35,3.678403e-33
4,GGACC,145,358,-11.258501,1.051526e-29,6.729769e-28
5,GGTCC,129,321,-10.717335,4.220010e-27,2.700807e-25
6,GCCGC,989,1298,-8.579761,4.753510e-18,3.042247e-16
7,CTCAG,525,736,-7.779128,3.651317e-15,2.336843e-13
8,CAGTG,1041,1318,-7.632706,1.149387e-14,7.356077e-13
9,ACCGT,1178,1469,-7.595502,1.533010e-14,9.811264e-13


In [ ]:
words = ['CCTGG', 'CCAGG', 'GAATC', 'GATTC', 'GGACC', 'GGTCC', 'GCCGC',
       'CTCAG', 'CAGTG', 'ACCGT', 'GCGGC', 'TCAGA', 'ACGGT', 'TATTA',
       'TCTGA', 'CTGAG', 'CACTG', 'TTGAA', 'TTCAA', 'GCTGC', 'TCCGA',
       'TAATA', 'TGTCA', 'AACTT', 'ACTGT', 'AAGTT', 'CGTCG', 'GCAGC',
       'ATTAT', 'TGACA', 'TTAAA', 'CGCCG', 'ATAAT', 'GACTC', 'CGGCG',
       'TAGTA', 'GAGTC', 'CGACG', 'TCGGA', 'TTTAA', 'CTAAG', 'AGCCT',
       'CCCGG', 'CCGGG', 'GGCCC', 'GGGCC']

In [ ]:
len(kmer_list)

In [ ]:
# grouped_mers = clustering_mers(words)

In [65]:
merge_expec = defaultdict(int)
merged_counts = defaultdict(int)
for key, value in clustered_pal.items():
    merge_expec[key] = merge_expec.get(key, 0)
    merged_counts[key] = merged_counts.get(key, 0)
    for kmer in value:
        merge_expec[key] += exp[kmer]
        merged_counts[key] += counts[kmer]
print(merge_expec)
print(merged_counts)

defaultdict(<class 'int'>, {'AACTT': 24525, 'ACAGT': 5189, 'AGACT': 3744, 'ATCAT': 16306, 'CACTG': 8546, 'CCGGG': 1723, 'CGACG': 3307, 'CTAAG': 3640, 'GAGTC': 4364, 'GCCGC': 5908, 'GGACC': 1015, 'GTCAC': 4358, 'TAGTA': 10922, 'TCGGA': 4680, 'TGACA': 8295, 'TTTAA': 24459})
defaultdict(<class 'int'>, {'AACTT': 23971, 'ACAGT': 4340, 'AGACT': 4059, 'ATCAT': 15254, 'CACTG': 8273, 'CCGGG': 956, 'CGACG': 2748, 'CTAAG': 3010, 'GAGTC': 3140, 'GCCGC': 4842, 'GGACC': 793, 'GTCAC': 4030, 'TAGTA': 9792, 'TCGGA': 3809, 'TGACA': 7947, 'TTTAA': 22725})


In [ ]:
x = df['kmer'].values
x

In [ ]:
y = df['z_score'].values
y

In [ ]:
plt.plot(x, y)
plt.xticks(rotation=90)
plt.axhline(y=0, color='r', linestyle='-')
plt.show();

In [ ]:
words = df['kmer'].values

In [ ]:
# def insert_one_wild_card(word):
#     mid = len(word)//2
#     if word[mid] in 'AT' and word[:mid] == get_reverse_complement( word[mid+1:]):
#         return word[:mid] +'W'+ word[mid+1:]
#     elif word[mid] in 'GC' and word[:mid] == get_reverse_complement( word[mid+1:]):
#         return word[:mid] +'S'+ word[mid+1:]

In [ ]:
# for word in words:
#     print(insert_one_wild_card(word))

In [ ]:
# dimers = [''.join(kmer) for kmer in itertools.product('ACGT',repeat=2)]

In [ ]:
# mononuc = {'A': 'W', 'C': 'S', 'G': 'S', 'T': 'W'}

In [ ]:
# dimers = [''.join(kmer) for kmer in itertools.product('ACGT',repeat=2)]
# wild_cards = ['WW', 'WS', 'WS', 'WW', 'SW', 'SS', 'SS', 'SW', 'SW', 'SS', 'SS', 'SW', 'WW', 'WS', 'WS', 'WW']

In [ ]:
# wild_cards_dimers = dict(zip(dimers, wild_cards))

In [ ]:
# trimers =  [''.join(kmer) for kmer in itertools.product('ACGT',repeat=3)]

In [ ]:
# def clustering_kmers(kmers_lst):
#     """Returns a set of aggruped kmers of length k."""
#     clusters = defaultdict(set)
#     for item in kmers_lst:
#         clusters[str(Counter(item))].add(item)
#     return clusters

In [ ]:
# with open('kmer_list_to_cluster.fa', 'w') as fho:
#     for i, kmer in enumerate(kmer_list, 1):
#         fho.write(f'>seq{i}\n{kmer}\n')   

In [ ]:
# def levenshtein_distance(s1, s2):
#     """Return the Levenshtein Distance for 2 strings.
#        Unequal string lengths are permitted for LD.
#     """
#     if len(s1) < len(s2):
#         return levenshtein_distance(s2, s1)

#     # len(s1) >= len(s2)
#     if len(s2) == 0:
#         return len(s1)

#     previous_row = range(len(s2) + 1)
#     for i, c1 in enumerate(s1):
#         current_row = [i + 1]
#         for j, c2 in enumerate(s2):
#             insertions = previous_row[j + 1] + 1
#             deletions = current_row[j] + 1
#             substitutions = previous_row[j] + (c1 != c2)
#             current_row.append(min(insertions, deletions, substitutions))
#         previous_row = current_row

#     return previous_row[-1]

In [ ]:
levenshtein_distance('CCTGG', 'CCAGG')

In [ ]:
hamming_distance('CCTGG', 'CCAGG')

In [ ]:
# def jaccard_similarity(word1, word2):
#     """Compute the Jaccard similarity of the sequence as kmer vectors
#     """
#     set1 = set(word1)
#     set2 = set(word2)
#     return len(set1.intersection(set2)) / len(set1.union(set2))

In [ ]:
jaccard_similarity('CCTGG', 'CCAGG')

In [ ]:
percent_id('CCTGG', 'CCAGG')

In [ ]:
def get_subpalindrome_count(wild_card_word, kmer_counts):
    """
    """
    word = wild_card_word.upper()
    mononuc = {'A': 'W', 
               'C': 'S', 
               'G': 'S', 
               'T': 'W'}
    wild_cards_dimers = {'AA': 'WW',  'AC': 'WS','AG': 'WS',
                         'AT': 'WW', 'CA': 'SW', 'CC': 'SS',
                         'CG': 'SS', 'CT': 'SW', 'GA': 'SW',
                         'GC': 'SS', 'GG': 'SS', 'GT': 'SW',
                         'TA': 'WW', 'TC': 'WS', 'TG': 'WS',
                         'TT': 'WW'}
    wild_cards_trimers = {'AAA': 'WWW', 'AAC': 'WWS', 'AAG': 'WWS',
                          'AAT': 'WWW', 'ACA': 'WSW', 'ACC': 'WSS',
                          'ACG': 'WSS', 'ACT': 'WSW', 'AGA': 'WSW',
                          'AGC': 'WSS', 'AGG': 'WSS', 'AGT': 'WSW',
                          'ATA': 'WWW', 'ATC': 'WWS', 'ATG': 'WWS',
                          'ATT': 'WWW', 'CAA': 'SWW', 'CAC': 'SWS',
                          'CAG': 'SWS', 'CAT': 'SWW', 'CCA': 'SSW',
                          'CCC': 'SSS', 'CCG': 'SSS', 'CCT': 'SSW',
                          'CGA': 'SSW', 'CGC': 'SSS', 'CGG': 'SSS',
                          'CGT': 'SSW', 'CTA': 'SWW', 'CTC': 'SWS',
                          'CTG': 'SWS', 'CTT': 'SWW', 'GAA': 'SWW',
                          'GAC': 'SWS', 'GAG': 'SWS', 'GAT': 'SWW',
                          'GCA': 'SSW', 'GCC': 'SSS', 'GCG': 'SSS',
                          'GCT': 'SSW', 'GGA': 'SSW', 'GGC': 'SSS',
                          'GGG': 'SSS', 'GGT': 'SSW', 'GTA': 'SWW',
                          'GTC': 'SWS', 'GTG': 'SWS', 'GTT': 'SWW',
                          'TAA': 'WWW', 'TAC': 'WWS', 'TAG': 'WWS',
                          'TAT': 'WWW', 'TCA': 'WSW', 'TCC': 'WSS',
                          'TCG': 'WSS', 'TCT': 'WSW', 'TGA': 'WSW',
                          'TGC': 'WSS', 'TGG': 'WSS', 'TGT': 'WSW',
                          'TTA': 'WWW', 'TTC': 'WWS', 'TTG': 'WWS',
                          'TTT': 'WWW'}
    #resolved_palindromes will store the resolved interpretations of 'palindrome', that is
    #both possibilities of a 'W' or 'S' character if present, or just 'palindrome' if no
    #'W' or 'S' is present. We can then iterate over resolved_palindromes to get the 
    #total count.
    resolved_palindromes = []
    count = 0
    if 'S' in word:
        #'S' stands for 'G' or 'C', so generate both of these possbilities.
        resolved_palindromes.append(string.replace(word,'S','C'))
        resolved_palindromes.append(string.replace(word,'S','G'))
    elif 'W' in word:
        #'W' stands for 'A' or 'T' so generate both possible interpretations.
        resolved_palindromes.append(string.replace(word,'W','A'))
        resolved_palindromes.append(string.replace(word,'W','T'))
    else: return kmer_counts[word]
    for resolved_palindrome in resolved_palindromes:
        count += kmer_counts[resolved_palindrome]
    return count

In [ ]:
4**3, 4**2

In [ ]:
rev_dimers = [(v, k) for k, v in wild_cards_dimers.items()]

In [ ]:
clustered = clustering_mers(words)

In [ ]:
percent_id('GGTCC', 'TGTCA')

In [ ]:
w = np.asarray(kmer_list)

In [ ]:
len(w)

In [ ]:
lev_similarity = - 1 * np.array([[distance.levenshtein(w1, w2) for w1 in w] for w2 in w])

In [ ]:
affprop = sklearn.cluster.AffinityPropagation(affinity="precomputed", damping=0.5, random_state=4)
affprop.fit(lev_similarity)

for cluster_id in np.unique(affprop.labels_):
    exemplar = w[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(w[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

In [ ]:
ndefaultdict(list,
            {'GCCGC': ['CCCGG, CCTGG, GCAGC, GCCGC, GCGGC, GCTGC'],
             'TCAGA': ['TCAGA, TCCGA, TCGGA, TCTGA'],
             'ACGGT': ['AAGTT, ACCGT, ACGGT, ACTGT, CCGGG'],
             'GACTC': ['AACTT, CACTG, GAATC, GACTC, GAGTC, GATTC'],
             'TAGTA': ['CAGTG, TAATA, TAGTA, TATTA'],
             'CGACG': ['CCAGG, CGACG, CGCCG, CGGCG, CGTCG, TGACA'],
             'TTTAA': ['ATTAT, TGTCA, TTAAA, TTCAA, TTGAA, TTTAA'],
             'CTAAG': ['ATAAT, CTAAG, CTCAG, CTGAG'],
             'GGCCC': ['AGCCT, GGACC, GGCCC, GGGCC, GGTCC']})

In [ ]:
import gensim

In [ ]:
data = compute_kmer_stats(kmer_list, counts_kmers, seq_len, 0.01)

In [ ]:
df_kmers = pd.DataFrame.from_records(data)

In [ ]:
df_kmers.rename(columns={0:'kmers', 
                   1:'obs', 
                   2:'expected', 
                   3:'z_score', 
                   4:'p_value', 
                   5:'e_value'}, 
          inplace=True)

In [ ]:
df_kmers['length'] = df_kmers['kmers'].str.len()

In [ ]:
df_kmers.sort_values(['length', 'kmers'], inplace=True)

In [ ]:
df_kmers.iloc[0:48]

In [ ]:
df_kmers_grouped_len = df_kmers.groupby('length')

In [ ]:
df_kmer_6 = df_kmers_grouped_len.get_group(6).sort_values(by=['z_score'])

In [ ]:
count_umbiguous_bases(sequence)

In [ ]:

def count_umbiguous_bases(sequence):
    sequence = sequence.upper()
    amb = ['N', 'R', 'Y', 'W', 'S', 'K', 'M']
    return sum({base: sequence.count(base) for base in amb}.values())


def base_stats(sequence, alphabet, as_count=False, as_dict=False):
    """Calculates de frequency or the number of bases in a sequence.
    Inputs:
    sequence - string representing the sequence
    alphabet - a alphabet (strings characters) that compound the string sequence
    as_count - boolean set as False
    as_dict - boolean set as False
    Output:
    counts - as default returns a numpy array as frequencies (floats)
    > baseFreqs(seq, 'ACGT', asCounts = False, asDict = False)
    array([0.25, 0.25, 0.25, 0.25])

    as_count - True, returns a numpy array of counts (integer)
    > baseFreqs('ACGTACGT', 'ACGT', asCounts = True, asDict = False)
    array([2, 2, 2, 2])

    as_dict - True and as_count as default (False) returns a dictionary as bases frequencies (float)
    > baseFreqs('ACGTACGT', 'ACGT', asCounts = False, asDict = True)
    {'A': 0.25, 'C': 0.25, 'G': 0.25, 'T': 0.25}

    as_count True and as_dict True, returns a dictionary as base counts (integer)
    > baseFreqs('ACGTACGT', 'ACGT', asCounts = True, asDict = True)
    {'A': 2, 'C': 2, 'G': 2, 'T': 2}
    """
    seq = sequence.upper()
    counts = np.array([seq.count(i) for i in alphabet])
    if as_count:
        freqs = counts
    else:
        freqs = counts / sum(counts * 1.0)
    if as_dict:
        return dict(zip(alphabet, freqs))
    else:
        return freqs

In [ ]:
bases = base_stats(sequence, 'ACGT', as_count=False, as_dict=True)

In [ ]:
# GTTAAC
probk = bases['G']*bases['T']*bases['T']*bases['A']*bases['A']*bases['C']
probk

In [ ]:
Ex = int((seq_len-count_umbiguous_bases(sequence))*probk) # 336
Ex, seq_len

In [ ]:
df_kmer_6.head()

In [ ]:
f = df_kmer_6['kmers'] == 'GTTAAC'
df_kmer_6.loc[f, 'expected'].values[0]

In [ ]:
df_kmers.loc[1, ['kmers', 'obs']].values[1]

In [ ]:
df_kmers_grouped_len.groups

In [ ]:
mers_6 = df_kmers_grouped_len.get_group(6).copy()

In [ ]:
mers_6['frequency'] = mers_6['obs'] / mers_6['obs'].sum()
mers_6['frequency'].sum()

In [ ]:
mers_6.reset_index(drop=True, inplace=True)

In [ ]:
mers_6.sort_values('kmers')

In [ ]:
c_kmers = count_mers(sequence, kmer_list, 'ACGT', 4, 6)

In [ ]:
c_kmers['GTTAAC']

In [ ]:
c_kmers_56 = {k:v for k,v in c_kmers.items() if v > 0 and len(k) == 5}

In [ ]:
c_kmers_56

In [ ]:
c_kmers_56_freq = {k:v/sum(c_kmers_56.values()) for k,v in c_kmers_56.items()}

In [ ]:
GTTAA = [c_kmers_56_freq['GTTAA'],
                          c_kmers_56_freq['GTTAC'],
                          c_kmers_56_freq['GTTAG'],
                          c_kmers_56_freq['GTTAT']]

In [ ]:
e = int((sum(GTTAA)*bases['C'])*(seq_len-6+1))

In [ ]:
e

In [ ]:
(19 - e)/e, (19-336)/336

In [ ]:
def sample(iterable, n):
    """ Samples n items from a stream """
    samp = []
    for t, item in enumerate(iterable):
        if t < n:
            samp.append(item)
        else:
            m = random.randint(0, t)
            if m < n:
                samp[m] = item
    return samp

In [ ]:
sample(sequence, 100)

In [280]:
def kmers_from_fasta(fh, k):
    """ Yield k-mer, offset pairs from FASTA filehandle.
        Ignore k-mers with chars besides A, C, G or T. """
    non_acgt = re.compile('[^ACGTacgt]')  # regex for detecting non-A/C/G/Ts
    kmer, off = [], 0
    for ln in fh:
        if ln[0] == r'>':
            kmer, off = [], 0  # new sequence
            continue
        for c in filter(lambda x: x.isalpha(), ln.decode()):
            if len(kmer) == k:
                kmer.pop(0)  # k-mer buffer full, so bump one element
            kmer.append(c.upper())
            off += 1
            if len(kmer) == k:
                kmerstr = ''.join(kmer)
                if not non_acgt.search(kmerstr):
                    yield kmerstr, off - k

In [281]:
list(kmers_from_fasta('Data/test/chromosomes/H.influenzae.fa.gz', 6))

TypeError: filter expected 2 arguments, got 1

In [294]:
def save_final_results(df, dir_out, spc, sub_sub_dir, sub_sub_sub_dir):
    full_name_dir = os.path.join(dir_out, spc, sub_sub_dir, sub_sub_sub_dir)
    csv_name = os.path.join(full_name_dir, name)
    if not os.path.exists(full_name_dir):
        os.makedirs(full_name_dir)
    df.sort_values('z_score', inplace=True)
    df.to_csv(f'{csv_name}_{sub_sub_sub_dir}_{max_k}.csv', index=False)

In [287]:
mer_2_counts = count_mers_cython(sequence_cleaner(sequence, 'ACGT'), 2, 2)

In [288]:
mer_2_counts

{'TA': 131964,
 'AT': 166845,
 'TG': 120001,
 'GG': 66449,
 'GC': 95532,
 'CA': 121629,
 'AA': 219894,
 'TT': 217523,
 'GT': 91319,
 'TC': 94753,
 'CG': 72525,
 'CC': 68016,
 'AG': 88461,
 'AC': 92423,
 'GA': 94136,
 'CT': 88554}

In [ ]:
dinuc = {             'AA': 219880,
             'AC': 92410,
             'AG': 88457,
             'AT': 166837,
             'CA': 121618,
             'CC': 68014,
             'CG': 72523,
             'CT': 88552,
             'GA': 94125,
             'GC': 95530,
             'GG': 66448,
             'GT': 91313,
             'TA': 131955,
             'TC': 94745,
             'TG': 119996,
             'TT': 217513}

In [289]:
def markov_chain_from_dinucs(dinucs):
    ''' Given dinucleotide frequencies, make a transition table. '''
    conds = np.zeros((4, 4), dtype=np.float64)
    margs = np.zeros(4, dtype=np.float64)
    for i, ci in enumerate('ACGT'):
        tot = 0
        for j, cj in enumerate('ACGT'):
            count = dinucs.get(ci + cj, 0)
            tot += count
            margs[i] += count
        if tot > 0:
            for j, cj in enumerate('ACGT'):
                conds[i, j] = dinucs.get(ci + cj, 0) / float(tot)
    return conds, margs

In [290]:
ins_conds, ins_margs = markov_chain_from_dinucs(mer_2_counts)

In [292]:
ins_margs

array([567623., 350724., 347436., 564241.])

In [293]:
np.sum(ins_conds, 1), np.log2(ins_conds)

(array([1., 1., 1., 1.]),
 array([[-1.36812481, -2.61860922, -2.68181959, -1.7664246 ],
        [-1.52784891, -2.36639009, -2.27378586, -1.98570678],
        [-1.88392879, -1.86269127, -2.38642785, -1.92776028],
        [-2.09616709, -2.57406798, -2.23326507, -1.37514355]]))

In [ ]:
class MarkovChainOrder(object):
    ''' Simple higher-order Markov chain, specifically for DNA
        sequences.  User provides training data (DNA strings).  '''
    
    def __init__(self, examples, order=1):
        ''' Initialize the model given collection of example DNA
            strings. '''
        self.order = order
        self.mers = defaultdict(int)
        self.longestMer = longestMer = order + 1
        for ex in examples:
            # count up all k-mers of length 'longestMer' or shorter
            for i in range(len(ex) - longestMer + 1):
                for j in range(longestMer, -1, -1):
                    self.mers[ex[i:i+j]] += 1
    
    def condProb(self, obs, given):
        ''' Return conditional probability of seeing nucleotide "obs"
            given we just saw nucleotide string "given".  Length of
            "given" can't exceed model order.  Return None if "given"
            was never observed. '''
        assert len(given) <= self.order
        ngiven = self.mers[given]
        if ngiven == 0: return None
        return float(self.mers[given + obs]) / self.mers[given]
    
    def jointProb(self, s):
        ''' Return joint probability of observing string s '''
        cum = 1.0
        for i in range(self.longestMer-1, len(s)):
            obs, given = s[i], s[i-self.longestMer+1:i]
            p = self.condProb(obs, given)
            if p is not None:
                cum *= p
        # include the smaller k-mers at the very beginning
        for j in range(self.longestMer-2, -1, -1):
            obs, given = s[j], s[:j]
            p = self.condProb(obs, given)
            if p is not None:
                cum *= p
        return cum
    
    def jointProbL(self, s):
        ''' Return log2 of joint probability of observing string s '''
        cum = 0.0
        for i in range(self.longestMer-1, len(s)):
            obs, given = s[i], s[i-self.longestMer+1:i]
            p = self.condProb(obs, given)
            if p is not None:
                cum += math.log(p, 2)
        # include the smaller k-mers at the very beginning
        for j in range(self.longestMer-2, -1, -1):
            obs, given = s[j], s[:j]
            p = self.condProb(obs, given)
            if p is not None:
                cum += math.log(p, 2)
        return cum

In [ ]:
mc3 = MarkovChainOrder(['GTTAAC'*10], 3)

In [ ]:
# prob cond onde A ocorre sempre apos TA = 1

mc3.condProb('A', 'TA')

In [ ]:
mc3.condProb('CA', 'TA') # não ocorre CA apos TA na sequencia

In [ ]:
mc3.condProb('AC', 'TA')# prob cond onde C ocorre sempre apos TA = 1

In [ ]:
mc3.condProb('TA', 'GT')# prob cond onde TA ocorre sempre apos GT = 1

In [ ]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits[:]

In [ ]:
def queryIndex(p, t, index):
    k = index.k
    offsets = []
    for i in index.query(p):
        if p[k:] == t[i+k:i+len(p)]:  # verify that rest of P matches
            offsets.append(i)
    return offsets

In [ ]:
t = 'ACTTGGAGATCTTTGAGGCTAGGTATTCGGGATCGAAGCTCATTTCGGGGATCGATTACGATATGGTGGGTATTCGGGA'
p = 'GGTATTCGGGA'

In [ ]:
index = Index(t, 4)
print(index.query(p))
print(queryIndex(p, t, index))

In [ ]:
t[21:21+len(p)], t[68:68+len(p)]

In [ ]:
def jaccard(Aset, Bset):
    # return Jaccard similarity coefficient between two sets
    isz = len(Aset.intersection(Bset))
    return float(isz) / (len(Aset) + len(Bset) - isz)

In [ ]:
def jaccard_kmer(Astr, Bstr, k):
    # turn two strings into sets, then return Jaccard similarity coefficient of those sets
    return jaccard(string_to_kmer_set(Astr, k),
                   string_to_kmer_set(Bstr, k))

In [ ]:
def string_to_kmer_set(Astr, k):
    return set([Astr[i:i+k] for i in range(len(Astr)-k+1)])

In [ ]:
jaccard_kmer("ABC", "ABD", 2)

In [ ]:
def add_mutations(string, num_muts):
    """ Add num_muts random substitution mutations to string """
    for _ in range(num_muts):
        rndi = random.randint(0, len(string)-1)
        string = string[:rndi] + random.choice('ACGT') + string[rndi+1:]
    return string

def random_jaccard_kmer(length, k):
    """ Make a random string and a second string separated from the
        first by a few mutations, then return the two strings and
        their jaccard similarity coefficient. """
    str1 = ''.join([random.choice('ACGT') for _ in range(length)])
    str2 = add_mutations(str1, random.randint(0, float(length)/20))
    return str1, str2, jaccard_kmer(str1, str2, k)

In [ ]:
random.seed(77)
add_mutations('ACGTACGT', 2)

In [ ]:
random.seed(76)
random_jaccard_kmer(20, 4)

In [ ]:
def string_to_min_kmer(Astr, k):
    return min([Astr[i:i+k] for i in range(len(Astr)-k+1)])

In [ ]:
def jaccard_min_kmer(Astr, Bstr, k):
    return 1 if string_to_min_kmer(Astr, k) == string_to_min_kmer(Bstr, k) else 0

In [ ]:
jaccard_min_kmer("ABC", "ABD", 2)

In [ ]:
def get_chunks(sequence, window, step=1):
    """Returns a chunk of length of window_size and the end of the window size"""
    sequence = sequence.upper()
    k = len(sequence)
    for i in range(0, k - window + 1, step):
        end = i + window
        chunk = sequence[i:end]
        assert len(chunk) == window
        yield chunk, i, end

In [ ]:
w = seq_len//480
w

In [ ]:
chunks = list(get_chunks(sequence, 5000, step=1000))

In [ ]:
chunks[0]

In [ ]:
def count_mers_in_chunks(sequence, alphabet, kmin, kmax):
    